In [ ]:
"""
To Debug the Decomposition



"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-14 05:12:03,389 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-14 05:12:03,391 - settings - Setting database.user to celiib
INFO - 2021-01-14 05:12:03,391 - settings - Setting database.password to newceliipass
INFO - 2021-01-14 05:12:03,394 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-14 05:12:03,395 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-14 05:12:03,407 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-14 05:12:03,692 - settings - Setting enable_python_native_blobs to True


In [3]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-14 05:12:03,771 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-14 05:12:03,772 - settings - Setting database.user to celiib
INFO - 2021-01-14 05:12:03,773 - settings - Setting database.password to newceliipass
INFO - 2021-01-14 05:12:03,774 - settings - Setting enable_python_native_blobs to True


Sleeping 1 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-14 05:12:04,037 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [4]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [5]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 4097


In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

# Segment That we are checking

In [7]:
"""
Ohter neurons to check:

864691135654122946
864691135583731320


"""
segment_id=864691135654122946

In [8]:
#1) Get the segment id from the key
description = "0_25"
print(f"\n\n----- Working on {segment_id}-------")
global_start = time.time()

#2) Get the decimated mesh
current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

#3) Get the somas info *************************** Need to change this when actually run *******************
somas = du.get_soma_mesh_list(segment_id) 
print(f"somas = {somas}")
#4) Run the preprocessing


total_neuron_process_time = time.time()



----- Working on 864691135654122946-------
somas = [[<trimesh.Trimesh(vertices.shape=(27986, 3), faces.shape=(61131, 3))>, <trimesh.Trimesh(vertices.shape=(23355, 3), faces.shape=(46447, 3))>], array([1078.9196, 1078.9196]), array([0.636, 0.857])]


# Getting the Glia and Nuclie Faces

In [9]:
#3b) Get the glia and nuclei information 
glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)

# Doing the decomposition function

In [10]:
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)

In [11]:
total_neuron_process_time = time.time()

print(f"\n--- Beginning preprocessing of {segment_id}---")
recovered_neuron = neuron.Neuron(
mesh = current_neuron_mesh,
somas = somas,
segment_id=segment_id,
description=description,
suppress_preprocessing_print=False,
suppress_output=False,
calculate_spines=False,
glia_faces=glia_faces,
nuclei_faces = nuclei_faces,
widths_to_calculate=[],
    

        )

print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


--- Beginning preprocessing of 864691135654122946---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(27986, 3), faces.shape=(61131, 3))>, <trimesh.Trimesh(vertices.shape=(23355, 3), faces.shape=(46447, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(27986, 3), faces.shape=(61131, 3))>, <trimesh.Trimesh(vertices.shape=(23355, 3), faces.shape=(46447, 3))>]
soma_mesh_list_centers = [array([612225.88424569, 655490.48805831, 953872.07398342]), array([697944.29344466, 631961.59936202, 931666.62221794])]
Getting Glia and Nuclei Pieces Subtracted Away 1.7978665828704834
 Splitting mesh after soma cancellation 6.967139720916748
# of split_meshes = 13
 Containing Mesh Indices 0.15577411651611328
 non_soma_touching_meshes 2.0742416381835938e-05
There were 12 pieces found after size threshold
 Finding insi

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [13:28<00:00, 808.28s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 931.715173959732
branches_touching_root = [15]
length of Graph = 212362
Working on path [21816. 21832. 21851. 21865. 21884.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [25092. 25110. 25123. 25127.]
path_degrees = [3, 2, 2, 3]
Working on path [59610. 59673. 59738. 59815. 59865.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [75131. 75152. 75181. 75213. 75228.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [96728. 96740. 96756. 96762. 96773. 96789. 96801. 96812.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [96860. 96925. 96982. 97031. 97094. 97141. 97194. 97217.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [113438. 113502. 113552. 113597. 113655. 113712. 113768.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [158187. 158226. 158275. 158320. 158323.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [195416. 195452. 195480.]
path_degrees = [3, 2, 3]
Working on path [212370. 195484. 195520.


AFTER face_lookup_resolved_test


...


AFTER face_lookup_resolved_test



Decomposing first pass: 1.4948134422302246
Attempting to use MeshAfterParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 0.00018739700317382812
Total time for MAP sublimb processing 3.337860107421875e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 7.104873657226562e-05
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.2447056770324707
sbv[0].reshape(-1,3) = [[612288.3 662040.8 947726.2]]
closest_sk_pt_coord BEFORE = [611804.34549495 662149.77227665 947388.5498449 ]
current_skeleton.shape = (1284, 2, 3)
node_for_stitch = 1284: [611804.34549495 662149.77227665 947388.5498449 ]
node_for_stitch AFTER = 1284: [611804.34549495 662149.77227665 947388.5498449 ]
possible_node_loc = [ 726 1284]
possible_node_loc AFTER = [ 726 1284]
curr_shortest_path = [1284]
Current stitch point was a branch or endpoint
c


28102 28103
For marked faces: None
empty_indices % = 0.0
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test



Total time for Skeletonization and Mesh Correspondence = 3729.4347150325775


 ----- Working on Stitching ----------
Status of Main limb stitch point moved = True
Total time for stitching floating pieces = 37.96224498748779
Total time for Concept Networks = 168.9348020553589
Total time for all mesh and skeletonization decomp = 3936.332790851593
No soma volume ratios so computing them now
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 4097 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/Poisson_temp/neuron_468644.off -o /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/Poisson_temp/neuron_468644_poisson.off -s /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/Poisson_temp/poisson_439458.mls
removed temporary input file: /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/Poisson_temp/neuron_468644.off
removed temporary output file: /notebooks/

In [12]:
recovered_neuron.save_compressed_neuron(output_folder="./",
                                       export_mesh=True)

Saving Neuorn in suppress_output mode...please wait
Saved File at location: 864691135654122946_0_25


In [13]:
neuron_obj = nru.decompress_neuron("./864691135654122946_0_25",
                                  "./864691135654122946_0_25")

Decompressing Neuron in minimal output mode...please wait


In [21]:
branch_obj = neuron_obj[0][0]

In [22]:
branch_obj.spines

[<trimesh.Trimesh(vertices.shape=(145, 3), faces.shape=(284, 3))>,
 <trimesh.Trimesh(vertices.shape=(134, 3), faces.shape=(263, 3))>,
 <trimesh.Trimesh(vertices.shape=(134, 3), faces.shape=(262, 3))>,
 <trimesh.Trimesh(vertices.shape=(113, 3), faces.shape=(221, 3))>,
 <trimesh.Trimesh(vertices.shape=(112, 3), faces.shape=(219, 3))>,
 <trimesh.Trimesh(vertices.shape=(107, 3), faces.shape=(209, 3))>,
 <trimesh.Trimesh(vertices.shape=(107, 3), faces.shape=(209, 3))>,
 <trimesh.Trimesh(vertices.shape=(107, 3), faces.shape=(208, 3))>,
 <trimesh.Trimesh(vertices.shape=(107, 3), faces.shape=(208, 3))>,
 <trimesh.Trimesh(vertices.shape=(106, 3), faces.shape=(207, 3))>,
 <trimesh.Trimesh(vertices.shape=(98, 3), faces.shape=(188, 3))>,
 <trimesh.Trimesh(vertices.shape=(94, 3), faces.shape=(182, 3))>,
 <trimesh.Trimesh(vertices.shape=(87, 3), faces.shape=(169, 3))>,
 <trimesh.Trimesh(vertices.shape=(84, 3), faces.shape=(163, 3))>,
 <trimesh.Trimesh(vertices.shape=(83, 3), faces.shape=(160, 3))>,


In [29]:
from pathlib import Path
import spine_utils as spu
spine_submesh_split= spu.get_spine_meshes_unfiltered(current_neuron = neuron_obj,
                                                                limb_idx=0,
                                                                branch_idx=0,
                                                                clusters=3,
                                                                smoothness=0.1,
                                                                cgal_folder = Path("./cgal_temp"),
                                                                delete_temp_file=True,
                                                                return_sdf=False,
                                                                print_flag=False,
                                                                shaft_threshold=300)

segment_name before cgal = 0_0


In [30]:
nviz.plot_objects(branch_obj.mesh,
                 meshes=spine_submesh_split,
                 meshes_colors="red")

In [24]:
nviz.plot_objects(branch_obj.mesh,
                 meshes=branch_obj.spines,
                 meshes_colors="red")

In [32]:
neuron = reload(neuron)
neuron_obj.calculate_spines(clusters_threshold=3,
                           smoothness_threshold=0.1)

query = median_mesh_center > 140 and n_faces_branch>100
smoothness_threshold = 0.1
The median_mesh_center was requested and HAS already been calculated
functions_list = [<function median_mesh_center at 0x7fedd84227b8>, <function n_faces_branch at 0x7fedd8422378>]
Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 2.725782 MB
segment_name before cgal = L0_0



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 0.515959 MB
segment_name before cgal = L0_2



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 1.78586 MB
segment_name before cgal = L0_9



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 0.869151 MB
segment_name before cgal = L0_11



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 1.886693 MB
segment_name before cgal = L0_12


...


Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 1.655927 MB
segment_name before cgal = L13_0



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 0.954453 MB
segment_name before cgal = L13_1



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 0.070161 MB
segment_name before cgal = L13_2



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 2.140015 MB
segment_name before cgal = L14_0



Saved object at /notebooks/Auto_Proofreading/Decomposition_Optimization/Debug_4/curr_branch_before_spines.pbz2
File size is 2.651215 MB
segment_name before cgal = L15_0


In [33]:
nviz.plot_spines(neuron_obj)


 Working on visualization type: mesh

 Working on visualization type: skeleton


# Running the Function to Decompose

In [ ]:
#predefined arguments for the Neuron constructor

decomposition_type="meshafterparty"
mesh_correspondence="meshparty" #meshafterparty_adaptive
distance_by_mesh_center=True #how the distance is calculated for mesh correspondence
meshparty_segment_size = 100
meshparty_n_surface_downsampling = 2
meshparty_adaptive_correspondence_after_creation=False
suppress_preprocessing_print=True
computed_attribute_dict=None


branch_skeleton_data=None
combine_close_skeleton_nodes = True
combine_close_skeleton_nodes_threshold=700

ignore_warnings=True
suppress_output=False
calculate_spines=True
widths_to_calculate=["no_spine_median_mesh_center"]
fill_hole_size=0# The old value for the parameter when performing 2000,

preprocessing_version=2
limb_to_branch_objects=None

In [ ]:
#arguments for the preprocess neuron
mesh = current_neuron_mesh
segment_id=segment_id
description=description

sig_th_initial_split=15 #for significant splitting meshes in the intial mesh split
limb_threshold = 2000 #the mesh faces threshold for a mesh to be qualified as a limb (otherwise too small)
filter_end_node_length=4500 #used in cleaning the skeleton during skeletonizations
return_no_somas = False

decomposition_type=decomposition_type
mesh_correspondence=mesh_correspondence
distance_by_mesh_center=distance_by_mesh_center
meshparty_segment_size =meshparty_segment_size
meshparty_n_surface_downsampling = meshparty_n_surface_downsampling
somas=somas
branch_skeleton_data=branch_skeleton_data
combine_close_skeleton_nodes = combine_close_skeleton_nodes
combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold

use_meshafterparty=True

# Starting of the preprocessing function

In [ ]:
from preprocessing_vp2 import *

In [ ]:
pre_branch_connectivity = "edges"
print(f"use_meshafterparty = {use_meshafterparty}")

whole_processing_tiempo = time.time()


"""
Purpose: To process the mesh into a format that can be loaded into the neuron class
and used for higher order processing (how to visualize is included)

This method includes the fusion

"""
if description is None:
    description = "no_description"
if segment_id is None:
    #pick a random segment id
    segment_id = np.random.randint(100000000)
    print(f"picking a random 7 digit segment id: {segment_id}")
    description += "_random_id"


if mesh is None:
    if mesh_file is None:
        raise Exception("No mesh or mesh_file file were given")
    else:
        current_neuron = tu.load_mesh_no_processing(mesh_file)
else:
    current_neuron = mesh


In [ ]:
# -------- Phase 1: Doing Soma Detection (if Not already done) ---------- #
if somas is None:
    soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)
else:
    soma_mesh_list,run_time,total_soma_list_sdf = somas
    print(f"Using pre-computed somas: soma_mesh_list = {soma_mesh_list}")

# geting the soma centers
if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id} so just one mesh")
    soma_mesh_list_centers = []
    if return_no_somas:
        return_value= soma_mesh_list_centers
    raise Exception("Processing of No Somas is not yet implemented yet")
else:
    #compute the soma centers
    print(f"Soma List = {soma_mesh_list}")

    soma_mesh_list_centers = sm.find_soma_centroids(soma_mesh_list)
    print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")

        

# Filtering Pieces and Eliminating Inside Soma Pieces

In [ ]:
#--- Phase 2: getting the soma submeshes that are connected to each soma and identifiying those that aren't 
# ------------------ (and eliminating any mesh pieces inside the soma) ------------------------

# -------- 11/13 Addition: Will remove the inside nucleus --------- #
interior_time = time.time()



if glia_pieces is None or nuclei_piece is None:
    main_mesh_total,glia_pieces,nuclei_piece = tu.remove_nuclei_and_glia_meshes(current_neuron,
                                                                   verbose=True)
print(f"Total time for removing interior = {time.time() - interior_time}")


#finding the mesh pieces that contain the soma
#splitting the current neuron into distinct pieces
split_time = time.time()
split_meshes = tu.split_significant_pieces(
                            main_mesh_total,
                            significance_threshold=sig_th_initial_split,
                            print_flag=False,
                            connectivity=pre_branch_connectivity)
print(f"Total time for splitting mesh = {time.time() - split_time}")

print(f"# total split meshes = {len(split_meshes)}")

#returns the index of the split_meshes index that contains each soma    
containing_mesh_indices = sm.find_soma_centroid_containing_meshes(soma_mesh_list,
                                        split_meshes)

# filtering away any of the inside floating pieces: 
non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                 if i not in list(containing_mesh_indices.values())]

#Adding the step that will filter away any pieces that are inside the soma
if len(non_soma_touching_meshes) > 0 and len(soma_mesh_list) > 0:
    """
    *** want to save these pieces that are inside of the soma***
    """

    non_soma_touching_meshes,inside_pieces = sm.filter_away_inside_soma_pieces(soma_mesh_list,non_soma_touching_meshes,
                                    significance_threshold=sig_th_initial_split,
                                    return_inside_pieces = True)

else:
    non_soma_touching_meshes = []
    inside_pieces=[]

#adding in the nuclei center to the inside pieces
inside_pieces += inside_nucleus_pieces


split_meshes # the meshes of the original mesh
containing_mesh_indices #the mapping of each soma centroid to the correct split mesh
soma_containing_meshes = sm.grouping_containing_mesh_indices(containing_mesh_indices)

soma_touching_meshes = [split_meshes[k] for k in soma_containing_meshes.keys()]


#     print(f"# of non soma touching seperate meshes = {len(non_soma_touching_meshes)}")
#     print(f"# of inside pieces = {len(inside_pieces)}")
print(f"\n-----Before filtering away multiple disconneted soma pieces-----")
print(f"# of soma containing seperate meshes = {len(soma_touching_meshes)}")
print(f"meshes with somas = {soma_containing_meshes}")

# ------ 11/15 Addition: Part 2.b 

"""
Pseudocode: 
1) Get the largest of the meshes with a soma (largest in soma_touching_meshes)
2) Save all other meshes not the largest in 
3) Overwrite the following variables:
    soma_mesh_list
    soma_containing_meshes
    soma_touching_meshes
    total_soma_list_sdf


"""
#1) Get the largest of the meshes with a soma (largest in soma_touching_meshes)
soma_containing_meshes_keys = np.array(list(soma_containing_meshes.keys()))
soma_touching_meshes = np.array([split_meshes[k] for k in soma_containing_meshes_keys])
largest_soma_touching_mesh_idx = soma_containing_meshes_keys[np.argmax([len(kk.faces) for kk in soma_touching_meshes])]

#2) Save all other meshes not the largest in 
not_processed_soma_containing_meshes_idx = np.setdiff1d(soma_containing_meshes_keys,[largest_soma_touching_mesh_idx])
not_processed_soma_containing_meshes = [split_meshes[k] for k in not_processed_soma_containing_meshes_idx]
print(f"Number of not_processed_soma_containing_meshes = {len(not_processed_soma_containing_meshes)}")

"""
3) Overwrite the following variables:
    soma_mesh_list
    soma_containing_meshes
    soma_touching_meshes
    total_soma_list_sdf

"""

somas_idx_to_process = soma_containing_meshes[largest_soma_touching_mesh_idx]
soma_mesh_list = [soma_mesh_list[k] for k in somas_idx_to_process]

soma_containing_meshes = {largest_soma_touching_mesh_idx:list(np.arange(0,len(soma_mesh_list)))}

soma_touching_meshes = [split_meshes[largest_soma_touching_mesh_idx]]

total_soma_list_sdf = total_soma_list_sdf[somas_idx_to_process]

print(f"\n-----After filtering away multiple disconneted soma pieces-----")
print(f"# of soma containing seperate meshes = {len(soma_touching_meshes)}")
print(f"meshes with somas = {soma_containing_meshes}")

In [ ]:
#--- Phase 3:  Soma Extraction was great (but it wasn't the original soma faces), so now need to get the original soma faces and the original non-soma faces of original pieces

"""
for each soma touching mesh get the following:
1) original soma meshes
2) significant mesh pieces touching these somas
3) The soma connectivity to each of the significant mesh pieces
-- later will just translate the 


Process: 

1) Final all soma faces (through soma extraction and then soma original faces function)
2) Subtact all soma faces from original mesh
3) Find all significant mesh pieces
4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all
   the available somas
Conclusion: Will have connectivity map


"""

soma_touching_mesh_data = dict()

for z,(mesh_idx, soma_idxes) in enumerate(soma_containing_meshes.items()):
    soma_touching_mesh_data[z] = dict()
    print(f"\n\n----Working on soma-containing mesh piece {z}----")

    #1) Final all soma faces (through soma extraction and then soma original faces function)
    current_mesh = split_meshes[mesh_idx]

    current_soma_mesh_list = [soma_mesh_list[k] for k in soma_idxes]

    current_time = time.time()
    mesh_pieces_without_soma = sm.subtract_soma(current_soma_mesh_list,current_mesh,
                                                significance_threshold=250,
                                               connectivity=pre_branch_connectivity)
    print(f"Total time for Subtract Soam = {time.time() - current_time}")
    current_time = time.time()

    mesh_pieces_without_soma_stacked = tu.combine_meshes(mesh_pieces_without_soma)

    # find the original soma faces of mesh
    soma_faces = tu.original_mesh_faces_map(current_mesh,mesh_pieces_without_soma_stacked,matching=False)
    print(f"Total time for Original_mesh_faces_map for mesh_pieces without soma= {time.time() - current_time}")
    current_time = time.time()
    soma_meshes = current_mesh.submesh([soma_faces],append=True,repair=False)

    # finding the non-soma original faces
    non_soma_faces = tu.original_mesh_faces_map(current_mesh,soma_meshes,matching=False)
    non_soma_stacked_mesh = current_mesh.submesh([non_soma_faces],append=True,repair=False)

    print(f"Total time for Original_mesh_faces_map for somas= {time.time() - current_time}")
    current_time = time.time()

    #4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all the available somas
    # get all the seperate mesh faces

    #How to seperate the mesh faces
    seperate_soma_meshes,soma_face_components = tu.split(soma_meshes,only_watertight=False,
                                                        connectivity=pre_branch_connectivity)
    #take the top largest ones depending how many were originally in the soma list
    seperate_soma_meshes = seperate_soma_meshes[:len(soma_mesh_list)]
    soma_face_components = soma_face_components[:len(soma_mesh_list)]

    soma_touching_mesh_data[z]["soma_meshes"] = seperate_soma_meshes




    # 3) Find all significant mesh pieces
    """
    Pseudocode: 
    a) Iterate through all of the somas and get the pieces that are connected
    b) Concatenate all the results into one list and order
    c) Filter away the mesh pieces that aren't touching and add to the floating pieces

    """
    sig_non_soma_pieces,insignificant_limbs = tu.split_significant_pieces(non_soma_stacked_mesh,significance_threshold=limb_threshold,
                                                     return_insignificant_pieces=True,
                                                                         connectivity=pre_branch_connectivity)

    # a) Filter these down to only those touching the somas
    all_conneted_non_soma_pieces = []
    for i,curr_soma in enumerate(seperate_soma_meshes):
        (connected_mesh_pieces,
         connected_mesh_pieces_vertices,
         connected_mesh_pieces_vertices_idx) = tu.mesh_pieces_connectivity(
                        main_mesh=current_mesh,
                        central_piece=curr_soma,
                        periphery_pieces = sig_non_soma_pieces,
                        return_vertices = True,
                        return_vertices_idx=True)
        all_conneted_non_soma_pieces.append(connected_mesh_pieces)

    #b) Iterate through all of the somas and get the pieces that are connected
    t_non_soma_pieces = np.concatenate(all_conneted_non_soma_pieces)

    #c) Filter away the mesh pieces that aren't touching and add to the floating pieces
    sig_non_soma_pieces = [s_t for hh,s_t in enumerate(sig_non_soma_pieces) if hh in t_non_soma_pieces]
    new_floating_pieces = [s_t for hh,s_t in enumerate(sig_non_soma_pieces) if hh not in t_non_soma_pieces]

    print(f"new_floating_pieces = {new_floating_pieces}")

    non_soma_touching_meshes += new_floating_pieces



    print(f"Total time for sig_non_soma_pieces= {time.time() - current_time}")
    current_time = time.time()

    soma_touching_mesh_data[z]["branch_meshes"] = sig_non_soma_pieces






    print(f"Total time for split= {time.time() - current_time}")
    current_time = time.time()



    soma_to_piece_connectivity = dict()
    soma_to_piece_touching_vertices = dict()
    soma_to_piece_touching_vertices_idx = dict()
    limb_root_nodes = dict()

    m_vert_graph = tu.mesh_vertex_graph(current_mesh)

    for i,curr_soma in enumerate(seperate_soma_meshes):
        (connected_mesh_pieces,
         connected_mesh_pieces_vertices,
         connected_mesh_pieces_vertices_idx) = tu.mesh_pieces_connectivity(
                        main_mesh=current_mesh,
                        central_piece=curr_soma,
                        periphery_pieces = sig_non_soma_pieces,
                        return_vertices = True,
                        return_vertices_idx=True)
        #print(f"soma {i}: connected_mesh_pieces = {connected_mesh_pieces}")
        soma_to_piece_connectivity[i] = connected_mesh_pieces

        soma_to_piece_touching_vertices[i] = dict()
        for piece_index,piece_idx in enumerate(connected_mesh_pieces):
            limb_root_nodes[piece_idx] = connected_mesh_pieces_vertices[piece_index][0]

            """ Old way of finding vertex connected components on a mesh without trimesh function
            #find the number of touching groups and save those 
            soma_touching_graph = m_vert_graph.subgraph(connected_mesh_pieces_vertices_idx[piece_index])
            soma_con_comp = [current_mesh.vertices[np.array(list(k)).astype("int")] for k in list(nx.connected_components(soma_touching_graph))]
            soma_to_piece_touching_vertices[i][piece_idx] = soma_con_comp
            """

            soma_to_piece_touching_vertices[i][piece_idx] = tu.split_vertex_list_into_connected_components(
                                                vertex_indices_list=connected_mesh_pieces_vertices_idx[piece_index],
                                                mesh=current_mesh, 
                                                vertex_graph=m_vert_graph, 
                                                return_coordinates=True
                                               )





#         border_debug = False
#         if border_debug:
#             print(f"soma_to_piece_connectivity = {soma_to_piece_connectivity}")
#             print(f"soma_to_piece_touching_vertices = {soma_to_piece_touching_vertices}")


    print(f"Total time for mesh_pieces_connectivity= {time.time() - current_time}")

    soma_touching_mesh_data[z]["soma_to_piece_connectivity"] = soma_to_piece_connectivity

print(f"# of insignificant_limbs = {len(insignificant_limbs)} with trimesh : {insignificant_limbs}")
print(f"# of not_processed_soma_containing_meshes = {len(not_processed_soma_containing_meshes)} with trimesh : {not_processed_soma_containing_meshes}")




# Lets have an alert if there was more than one soma disconnected meshes
if len(soma_touching_mesh_data.keys()) > 1:
    raise Exception("More than 1 disconnected meshes that contain somas")

current_mesh_data = soma_touching_mesh_data
soma_containing_idx = 0

#doing inversion of the connectivity and touching vertices
piece_to_soma_touching_vertices = gu.flip_key_orders_for_dict(soma_to_piece_touching_vertices)



# Doing the Actual Limb Processing

In [ ]:
pre = reload(pre)

In [ ]:
# Phase 4: Skeletonization, Mesh Correspondence,  

proper_time = time.time()

#The containers that will hold the final data for the preprocessed neuron
limb_correspondence=dict()
limb_network_stating_info = dict()

# ---------- Part A: skeletonization and mesh decomposition --------- #
skeleton_time = time.time()

for curr_limb_idx,limb_mesh_mparty in enumerate(current_mesh_data[0]["branch_meshes"]):

    #Arguments to pass to the specific function (when working with a limb)
    soma_touching_vertices_dict = piece_to_soma_touching_vertices[curr_limb_idx]

    curr_limb_time = time.time()
    print(f"\n\n----- Working on Proper Limb # {curr_limb_idx} ---------")
    
    if curr_limb_idx != 1:
        continue
    else:
        raise Exception("")

    print(f"meshparty_segment_size = {meshparty_segment_size}")
    limb_correspondence_individual,network_starting_info = pre.preprocess_limb(mesh=limb_mesh_mparty,
                   soma_touching_vertices_dict = soma_touching_vertices_dict,
                   return_concept_network = False, 
                   return_concept_network_starting_info=True,
                   width_threshold_MAP=500,
                   size_threshold_MAP=2000,
                   surface_reconstruction_size=1000,  

                   #arguments added from the big preprocessing step                                                            
                   distance_by_mesh_center=distance_by_mesh_center,
                   meshparty_segment_size=meshparty_segment_size,
                   meshparty_n_surface_downsampling = meshparty_n_surface_downsampling,

                    use_meshafterparty=use_meshafterparty,

                   )
    #Storing all of the data to be sent to 

    limb_correspondence[curr_limb_idx] = limb_correspondence_individual
    limb_network_stating_info[curr_limb_idx] = network_starting_info
    
    raise Exception("Finished")

print(f"Total time for Skeletonization and Mesh Correspondence = {time.time() - skeleton_time}")



# ---------- Part B: Stitching on floating pieces --------- #
print("\n\n ----- Working on Stitching ----------")

floating_stitching_time = time.time()

if len(limb_correspondence) > 0:
    non_soma_touching_meshes_to_stitch = tu.check_meshes_outside_multiple_mesh_bbox(seperate_soma_meshes,non_soma_touching_meshes,
                             return_indices=False)

    limb_correspondence_with_floating_pieces = pre.attach_floating_pieces_to_limb_correspondence(
            limb_correspondence,
            floating_meshes=non_soma_touching_meshes_to_stitch,
            floating_piece_face_threshold = 600,
            max_stitch_distance=8000,
            distance_to_move_point_threshold = 4000,
            verbose = False)
else:
    limb_correspondence_with_floating_pieces = limb_correspondence




print(f"Total time for stitching floating pieces = {time.time() - floating_stitching_time}")





# ---------- Part C: Computing Concept Networks --------- #
concept_network_time = time.time()

limb_concept_networks=dict()
limb_labels=dict()

for curr_limb_idx,limb_mesh_mparty in enumerate(current_mesh_data[0]["branch_meshes"]):
    limb_to_soma_concept_networks = pre.calculate_limb_concept_networks(limb_correspondence_with_floating_pieces[curr_limb_idx],
                                                                    limb_network_stating_info[curr_limb_idx],
                                                                    run_concept_network_checks=True,
                                                                       )   



    limb_concept_networks[curr_limb_idx] = limb_to_soma_concept_networks
    limb_labels[curr_limb_idx]= "Unlabeled"

print(f"Total time for Concept Networks = {time.time() - concept_network_time}")



In [ ]:
curr_limb_idx

# Debugging the can't find endpoints


# The Limb Preprocessing Function

In [ ]:
mesh=limb_mesh_mparty
soma_touching_vertices_dict = soma_touching_vertices_dict
return_concept_network = False
return_concept_network_starting_info=True
width_threshold_MAP=500
size_threshold_MAP=2000
surface_reconstruction_size=1000  

#arguments added from the big preprocessing step                                                            
distance_by_mesh_center=distance_by_mesh_center
meshparty_segment_size=meshparty_segment_size
meshparty_n_surface_downsampling = meshparty_n_surface_downsampling

use_meshafterparty=use_meshafterparty




combine_close_skeleton_nodes=True
combine_close_skeleton_nodes_threshold=700
filter_end_node_length=4000
perform_cleaning_checks = True



#parameters for MP skeletonization,

#Parameters for setting how the MAP skeletonization takes place
use_surface_after_CGAL=False

#parametrers for stitching the MAP and MP pieces together
move_MAP_stitch_to_end_or_branch = True
distance_to_move_point_threshold=500

#concept_network parameters
run_concept_network_checks = True


#printing controls
verbose = True
print_fusion_steps=True

check_correspondence_branches = True
filter_end_nodes_from_correspondence = True

error_on_no_starting_coordinates=True,
prevent_MP_starter_branch_stitches = False, #will control if a MP soma extending branch is able to be stitched to
    



In [ ]:
curr_limb_time = time.time()

limb_mesh_mparty = mesh


#will store a list of all the endpoints tha tmust be kept:
limb_to_endpoints_must_keep_list = []
limb_to_soma_touching_vertices_list = []

# --------------- Part 1 and 2: Getting Border Vertices and Setting the Root------------- #
fusion_time = time.time()
#will eventually get the current root from soma_to_piece_touching_vertices[i]
if not soma_touching_vertices_dict is None:
    root_curr = soma_touching_vertices_dict[list(soma_touching_vertices_dict.keys())[0]][0][0]
else:
    root_curr = None

print(f"root_curr = {root_curr}")

if print_fusion_steps:
    print(f"Time for preparing soma vertices and root: {time.time() - fusion_time }")
    fusion_time = time.time()

# --------------- Part 3: Meshparty skeletonization and Decomposition ------------- #
sk_meshparty_obj = m_sk.skeletonize_mesh_largest_component(limb_mesh_mparty,
                                                        root=root_curr,
                                                          filter_mesh=False)

print(f"meshparty_segment_size = {meshparty_segment_size}")

if print_fusion_steps:
    print(f"Time for 1st pass MP skeletonization: {time.time() - fusion_time }")
    fusion_time = time.time()

(segment_branches, #skeleton branches
divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
segment_widths_median) = m_sk.skeleton_obj_to_branches(sk_meshparty_obj,
                                                      mesh = limb_mesh_mparty,
                                                      meshparty_segment_size=meshparty_segment_size)





if print_fusion_steps:
    print(f"Decomposing first pass: {time.time() - fusion_time }")
    fusion_time = time.time()


if use_meshafterparty:
    print("Attempting to use MeshAfterParty Skeletonization and Mesh Correspondence")
    # --------------- Part 4: Find Individual Branches that could be MAP processed because of width ------------- #
    #gettin the branches that should be passed through MAP skeletonization
    pieces_above_threshold = np.where(segment_widths_median>width_threshold_MAP)[0]

    #getting the correspondnece info for those MAP qualifying
    width_large = segment_widths_median[pieces_above_threshold]
    sk_large = [segment_branches[k] for k in pieces_above_threshold]
    mesh_large_idx = [divided_submeshes_idx[k] for k in pieces_above_threshold]
else:
    print("Only Using MeshParty Skeletonization and Mesh Correspondence")
    mesh_large_idx = []
    width_large = []
    sk_large = []


print("Another print")
mesh_pieces_for_MAP = []
mesh_pieces_for_MAP_face_idx = []


if len(mesh_large_idx) > 0: #will only continue processing if found MAP candidates

    # --------------- Part 5: Find mesh connectivity and group MAP branch candidates into MAP sublimbs ------------- #
    print(f"Found len(mesh_large_idx) MAP candidates: {[len(k) for k in mesh_large_idx]}")

    #finds the connectivity edges of all the MAP candidates
    mesh_large_connectivity = tu.mesh_list_connectivity(meshes = mesh_large_idx,
                                                        connectivity="edges",
                            main_mesh = limb_mesh_mparty,
                            print_flag = False)
    if print_fusion_steps:
        print(f"mesh_large_connectivity: {time.time() - fusion_time }")
        fusion_time = time.time()
        
    
    """
    --------------- Grouping MAP candidates ----------------
    Purpose: Will see what mesh pieces should be grouped together
    to pass through CGAL skeletonization


    Pseudocode: 
    1) build a networkx graph with all nodes for mesh_large_idx indexes
    2) Add the edges
    3) Find the connected components
    4) Find sizes of connected components
    5) For all those connected components that are of a large enough size, 
    add the mesh branches and skeletons to the final list


    """
    G = nx.Graph()
    G.add_nodes_from(np.arange(len(mesh_large_idx)))
    G.add_edges_from(mesh_large_connectivity)
    conn_comp = list(nx.connected_components(G))
    
    print("**** Debugging big mesh connectivity ********")
    print(f"mesh_large_connectivity = {mesh_large_connectivity}")
    print(f"conn_comp = {conn_comp}")

    filtered_pieces = []

    sk_large_size_filt = []
    mesh_large_idx_size_filt = []
    width_large_size_filt = []

    for cc in conn_comp:
        total_cc_size = np.sum([len(mesh_large_idx[k]) for k in cc])
        if total_cc_size>size_threshold_MAP:
            #print(f"cc ({cc}) passed the size threshold because size was {total_cc_size}")
            filtered_pieces.append(pieces_above_threshold[list(cc)])

    if print_fusion_steps:
        print(f"Finding MAP candidates connected components: {time.time() - fusion_time }")
        fusion_time = time.time()

    #filtered_pieces: will have the indexes of all the branch candidates that should  be 
    #grouped together and passed through MAP skeletonization

    if len(filtered_pieces) > 0:
        # --------------- Part 6: If Found MAP sublimbs, Get the meshes and mesh_idxs of the sublimbs ------------- #
        print(f"len(filtered_pieces) = {len(filtered_pieces)}")
        #all the pieces that will require MAP mesh correspondence and skeletonization
        #(already organized into their components)
        mesh_pieces_for_MAP = [limb_mesh_mparty.submesh([np.concatenate(divided_submeshes_idx[k])],append=True,repair=False) for k in filtered_pieces]
        mesh_pieces_for_MAP_face_idx = [np.concatenate(divided_submeshes_idx[k]) for k in filtered_pieces]



        """
        Old Way: Finding connectivity of pieces through
        mesh_idx_MP = [divided_submeshes_idx[k] for k in pieces_idx_MP]

        mesh_large_connectivity_MP = tu.mesh_list_connectivity(meshes = mesh_idx_MP,
                                main_mesh = limb_mesh_mparty,
                                print_flag = False)

        New Way: going to use skeleton connectivity to determine
        connectivity of pieces

        Pseudocode: 
        1)

        """
        # --------------- Part 7: If Found MAP sublimbs, Get the meshes and mesh_idxs of the sublimbs ------------- #
        # ********* if there are no pieces leftover then will automatically make all the lists below just empty (don't need to if.. else.. the case)****
        pieces_idx_MP = np.setdiff1d(np.arange(len(divided_submeshes_idx)),np.concatenate(filtered_pieces))

        skeleton_MP = [segment_branches[k] for k in pieces_idx_MP]
        skeleton_connectivity_MP = sk.skeleton_list_connectivity(
                                        skeletons=skeleton_MP
                                        )
        if print_fusion_steps:
            print(f"skeleton_connectivity_MP : {time.time() - fusion_time }")
            fusion_time = time.time()

        G = nx.Graph()
        G.add_nodes_from(np.arange(len(skeleton_MP)))
        G.add_edges_from(skeleton_connectivity_MP)
        sublimbs_MP = list(nx.connected_components(G))
        sublimbs_MP_orig_idx = [pieces_idx_MP[list(k)] for k in sublimbs_MP]


        #concatenate into sublimbs the skeletons and meshes
        sublimb_mesh_idx_branches_MP = [divided_submeshes_idx[k] for k in sublimbs_MP_orig_idx]
        sublimb_mesh_branches_MP = [[limb_mesh_mparty.submesh([ki],append=True,repair=False)
                                    for ki in k] for k in sublimb_mesh_idx_branches_MP]
        sublimb_meshes_MP = [limb_mesh_mparty.submesh([np.concatenate(k)],append=True,repair=False)
                                                     for k in sublimb_mesh_idx_branches_MP]
        sublimb_meshes_MP_face_idx = [np.concatenate(k)
                                                     for k in sublimb_mesh_idx_branches_MP]
        sublimb_skeleton_branches = [segment_branches[k] for k in sublimbs_MP_orig_idx]
        widths_MP = [segment_widths_median[k] for k in sublimbs_MP_orig_idx]

        if print_fusion_steps:
            print(f"Grouping MP Sublimbs by Graph: {time.time() - fusion_time }")
            fusion_time = time.time()


# else: #if no pieces were determine to need MAP processing
#     print("No MAP processing needed: just returning the Meshparty skeletonization and mesh correspondence")
#     raise Exception("Returning MP correspondence")


# nviz.plot_objects(main_mesh=tu.combine_meshes([limb_mesh_mparty,current_neuron["S0"].mesh]),
#                   main_mesh_color="green",
#     skeletons=sk_large_size_filt,
#      meshes=[limb_mesh_mparty.submesh([k],append=True) for k in mesh_large_idx_size_filt],
#       meshes_colors="red")








# --------------- Part 8: If No MAP sublimbs found, set the MP sublimb lists to just the whole MP branch decomposition ------------- #

#if no sublimbs need to be decomposed with MAP then just reassign all of the previous MP processing to the sublimb_MPs
if len(mesh_pieces_for_MAP) == 0:
    sublimb_meshes_MP = [limb_mesh_mparty] #trimesh pieces that have already been passed through MP skeletonization (may not need)
    # -- the decomposition information ---
    sublimb_mesh_branches_MP = [divided_submeshes] #the mesh branches for all the disconnected sublimbs
    sublimb_mesh_idx_branches_MP = [divided_submeshes_idx] #The mesh branches idx that have already passed through MP skeletonization
    sublimb_skeleton_branches = [segment_branches]#the skeleton bnraches for all the sublimbs
    widths_MP = [segment_widths_median] #the mesh branches widths for all the disconnected groups

    MAP_flag = False
else:
    MAP_flag = True



mesh_pieces_for_MAP #trimesh pieces that should go through CGAL skeletonization
sublimb_meshes_MP #trimesh pieces that have already been passed through MP skeletonization (may not need)

# -- the decomposition information ---
sublimb_mesh_branches_MP #the mesh branches for all the disconnected sublimbs
sublimb_mesh_idx_branches_MP #The mesh branches idx that have already passed through MP skeletonization
sublimb_skeleton_branches #the skeleton bnraches for all the sublimbs
widths_MP #the mesh branches widths for all the disconnected groups

if print_fusion_steps:
    print(f"Divinding into MP and MAP pieces: {time.time() - fusion_time }")
    fusion_time = time.time()



# ------------------- At this point have the correct division between MAP and MP ------------------------

# -------------- Part 9: Doing the MAP decomposition ------------------ #
global_start_time = time.time()
endpoints_must_keep = dict()



limb_correspondence_MAP = dict()

for sublimb_idx,(mesh,mesh_idx) in enumerate(zip(mesh_pieces_for_MAP,mesh_pieces_for_MAP_face_idx)):
    print(f"--- Working on MAP piece {sublimb_idx}---")
    #print(f"soma_touching_vertices_dict = {soma_touching_vertices_dict}")
    mesh_start_time = time.time()
    curr_soma_to_piece_touching_vertices = filter_soma_touching_vertices_dict_by_mesh(
    mesh = mesh,
    curr_piece_to_soma_touching_vertices = soma_touching_vertices_dict
    )

    if print_fusion_steps:
        print(f"MAP Filtering Soma Pieces: {time.time() - fusion_time }")
        fusion_time = time.time()

    # ---- 0) Generating the Clean skeletons  -------------------------------------------#
    if not curr_soma_to_piece_touching_vertices is None:
        curr_total_border_vertices = dict([(k,np.vstack(v)) for k,v in curr_soma_to_piece_touching_vertices.items()])
    else:
        curr_total_border_vertices = None


    cleaned_branch,curr_limb_endpoints_must_keep = sk.skeletonize_and_clean_connected_branch_CGAL(
        mesh=mesh,
        curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices,
        total_border_vertices=curr_total_border_vertices,
        filter_end_node_length=filter_end_node_length,
        perform_cleaning_checks=perform_cleaning_checks,
        combine_close_skeleton_nodes = combine_close_skeleton_nodes,
        combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold,
    use_surface_after_CGAL=use_surface_after_CGAL,
    surface_reconstruction_size=surface_reconstruction_size)

    if not curr_limb_endpoints_must_keep is None:
        limb_to_endpoints_must_keep_list.append(curr_limb_endpoints_must_keep)
        limb_to_soma_touching_vertices_list.append(curr_soma_to_piece_touching_vertices)
    else:
        print("Inside MAP decomposition and curr_limb_endpoints_must_keep was None")

    if len(cleaned_branch) == 0:
        raise Exception(f"Found a zero length skeleton for limb {z} of trmesh {branch}")

    if print_fusion_steps:
        print(f"skeletonize_and_clean_connected_branch_CGAL: {time.time() - fusion_time }")
        fusion_time = time.time()

    # ---- 1) Generating Initial Mesh Correspondence -------------------------------------------#
    start_time = time.time()

    print(f"Working on limb correspondence for #{sublimb_idx} MAP piece")
    local_correspondence = mesh_correspondence_first_pass(mesh=mesh,
                                                         skeleton=cleaned_branch,
                                                         distance_by_mesh_center=distance_by_mesh_center,
                                                         connectivity="edges",
                                                         remove_inside_pieces_threshold=100)


    print(f"Total time for decomposition = {time.time() - start_time}")
    if print_fusion_steps:
        print(f"mesh_correspondence_first_pass: {time.time() - fusion_time }")
        fusion_time = time.time()


    #------------- 2) Doing Some checks on the initial corespondence -------- #


    if perform_cleaning_checks:
        check_skeletonization_and_decomp(skeleton=cleaned_branch,
                                        local_correspondence=local_correspondence)

    # -------3) Finishing off the face correspondence so get 1-to-1 correspondence of mesh face to skeletal piece
    local_correspondence_revised = correspondence_1_to_1(mesh=mesh,
                                    local_correspondence=local_correspondence,
                                    curr_limb_endpoints_must_keep=curr_limb_endpoints_must_keep,
                                    curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices)

    # -------3b) Fixing the mesh indices to correspond to the larger mesh as a whole
    for k,v in local_correspondence_revised.items():
        local_correspondence_revised[k]["branch_face_idx"] = mesh_idx[local_correspondence_revised[k]["branch_face_idx"]]

    print(f"Total time for MAP sublimb #{sublimb_idx} mesh processing = {time.time() - mesh_start_time}")

    if print_fusion_steps:
        print(f"correspondence_1_to_1: {time.time() - fusion_time }")
        fusion_time = time.time()

    limb_correspondence_MAP[sublimb_idx] = local_correspondence_revised

print(f"Total time for MAP sublimb processing {time.time() - global_start_time}")





# ----------------- Part 10: Doing the MP Decomposition ---------------------- #




sublimb_meshes_MP #trimesh pieces that have already been passed through MP skeletonization (may not need)
# -- the decomposition information ---
sublimb_mesh_branches_MP #the mesh branches for all the disconnected sublimbs
sublimb_mesh_idx_branches_MP #The mesh branches idx that have already passed through MP skeletonization
sublimb_skeleton_branches #the skeleton bnraches for all the sublimbs
widths_MP #the mesh branches widths for all the disconnected groups

limb_correspondence_MP = dict()

for sublimb_idx,mesh in enumerate(sublimb_meshes_MP):
    print(f"---- Working on MP Decomposition #{sublimb_idx} ----")
    mesh_start_time = time.time()

    if len(sublimb_meshes_MP) == 1 and MAP_flag == False:
        print("Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece ")
        curr_soma_to_piece_touching_vertices = soma_touching_vertices_dict
    else:
        if not soma_touching_vertices_dict is None:
            print("Computing the current soma touching verts dict manually")
            curr_soma_to_piece_touching_vertices = filter_soma_touching_vertices_dict_by_mesh(
                                                mesh = mesh,
                                                curr_piece_to_soma_touching_vertices = soma_touching_vertices_dict
                                                )
        else:
            curr_soma_to_piece_touching_vertices = None

    if print_fusion_steps:
        print(f"MP filtering soma verts: {time.time() - fusion_time }")
        fusion_time = time.time()

    #creating all of the sublimb groups
    segment_branches = np.array(sublimb_skeleton_branches[sublimb_idx])
    whole_sk_MP = sk.stack_skeletons(segment_branches)
    branch = mesh
    divided_submeshes = np.array(sublimb_mesh_branches_MP[sublimb_idx])
    divided_submeshes_idx = sublimb_mesh_idx_branches_MP[sublimb_idx]
    segment_widths_median = widths_MP[sublimb_idx]


    if curr_soma_to_piece_touching_vertices is None:
        print(f"Do Not Need to Fix MP Decomposition {sublimb_idx} so just continuing")

    else:

        # ------- 11/9 addition: Fixing error where creating soma touching branch on mesh that doesn't touch border ------------------- #
        print(f"Fixing Possible Soma Extension Branch for Sublimb {sublimb_idx}")
        no_soma_extension_add = True 

        endpts_total = dict()
        curr_soma_to_piece_touching_vertices_total = dict()
        for sm_idx,sm_bord_verts_list in curr_soma_to_piece_touching_vertices.items():
            #will be used for later
            endpts_total[sm_idx] = []
            curr_soma_to_piece_touching_vertices_total[sm_idx] = []

            for sm_bord_verts in sm_bord_verts_list:
                #1) Get the mesh pieces that are touching the border
                matching_mesh_idx = tu.filter_meshes_by_containing_coordinates(mesh_list=divided_submeshes,
                                           nullifying_points=sm_bord_verts,
                                            filter_away=False,
                                           distance_threshold=0,
                                           return_indices=True)
                #2) concatenate all meshes and skeletons that are touching
                if len(matching_mesh_idx) <= 0:
                    raise Exception("None of branches were touching the border vertices when fixing MP pieces")

                touch_mesh = tu.combine_meshes(divided_submeshes[matching_mesh_idx])
                touch_sk = sk.stack_skeletons(segment_branches[matching_mesh_idx])

                local_curr_soma_to_piece_touching_vertices = {sm_idx:[sm_bord_verts]}
                new_sk,endpts,new_branch_info = sk.create_soma_extending_branches(current_skeleton=touch_sk,
                                      skeleton_mesh=touch_mesh,
                                      soma_to_piece_touching_vertices=local_curr_soma_to_piece_touching_vertices,
                                      return_endpoints_must_keep=True,
                                      return_created_branch_info=True,
                                      check_connected_skeleton=False)
                # ---- 12/30 Addition Check if the endpoint found is an endnode or not and if not then manually add branch ---
                curr_endnode = endpts[sm_idx][0]
                match_sk_branches = sk.find_branch_skeleton_with_specific_coordinate(segment_branches,
                    current_coordinate=curr_endnode)
                
                if len(match_sk_branches) > 1:
                    border_average_coordinate = np.mean(sm_bord_verts,axis=0)
                    new_branch_sk = np.vstack([curr_endnode,border_average_coordinate]).reshape(-1,2,3)
                    br_info = dict(new_branch = new_branch_sk,border_verts=sm_bord_verts)
                    endpts_total[sm_idx].append(border_average_coordinate)
                else:
                    br_info = new_branch_info[sm_idx][0]
                    endpts_total[sm_idx].append(endpts[sm_idx][0])
                # -------------------- End of 12/30 Addition ------------------
                

                #3) Add the info to the new running lists
                
                curr_soma_to_piece_touching_vertices_total[sm_idx].append(sm_bord_verts)


                #4) Skip if no new branch was added
                
                if br_info is None:
                    print("The new branch info was none so skipping \n")
                    continue

                #4 If new branch was made then 
                no_soma_extension_add=False

                #1) Get the newly added branch (and the original vertex which is the first row)
                br_new,sm_bord_verts = br_info["new_branch"],br_info["border_verts"] #this will hold the new branch and the border vertices corresponding to it

                curr_soma_to_piece_touching_vertices_MP = {sm_idx:[sm_bord_verts]}
                endpoints_must_keep_MP = {sm_idx:[br_new[0][1]]}


                orig_vertex = br_new[0][0]
                print(f"orig_vertex = {orig_vertex}")

                #2) Find the branches that have that coordinate (could be multiple)
                match_sk_branches = sk.find_branch_skeleton_with_specific_coordinate(segment_branches,
                    current_coordinate=orig_vertex)

                print(f"match_sk_branches = {match_sk_branches}")



                """ ******************* THIS NEEDS TO BE FIXED WITH THE SAME METHOD OF STITCHING ********************  """
                """
                Pseudocode:
                1) Find if branch point will require split or not
                2) If does require split then split the skeleton
                3) Gather mesh pieces for correspondence and the skeletons
                4) Run the mesh correspondence
                - this case calculate the new widths after run 
                5) Replace the old branch parts with the new ones



                """

                stitch_point_on_end_or_branch = find_if_stitch_point_on_end_or_branch(
                                                        matched_branches_skeletons= segment_branches[match_sk_branches],
                                                         stitch_coordinate=orig_vertex,
                                                          verbose=False)


                if not stitch_point_on_end_or_branch:
                    matching_branch_sk = sk.cut_skeleton_at_coordinate(skeleton=segment_branches[match_sk_branches][0],
                                                                      cut_coordinate = orig_vertex)
                else:
                    matching_branch_sk = segment_branches[match_sk_branches]


                #3) Find the mesh and skeleton of the winning branch
                matching_branch_meshes = np.array(divided_submeshes)[match_sk_branches]
                matching_branch_mesh_idx = np.array(divided_submeshes_idx)[match_sk_branches]
                extend_soma_mesh_idx = np.concatenate(matching_branch_mesh_idx)
                extend_soma_mesh = limb_mesh_mparty.submesh([extend_soma_mesh_idx ],append=True,repair=False)

                #4) Add newly created branch to skeleton and divide the skeleton into branches (could make 2 or 3)
                #extended_skeleton_to_soma = sk.stack_skeletons([list(matching_branch_sk),br_new])

                sk.check_skeleton_connected_component(sk.stack_skeletons(list(matching_branch_sk) + [br_new]))

                #5) Run Adaptive mesh correspondnece using branches and mesh
                local_correspondnece_MP = mesh_correspondence_first_pass(mesh=extend_soma_mesh,
                                                                         skeleton_branches = list(matching_branch_sk) + [br_new]
                                              #skeleton=extended_skeleton_to_soma
                                                                        )

                # GETTING MESHES THAT ARE NOT FULLY CONNECTED!!
                local_correspondence_revised = correspondence_1_to_1(mesh=extend_soma_mesh,
                                                            local_correspondence=local_correspondnece_MP,
                                                            curr_limb_endpoints_must_keep=endpoints_must_keep_MP,
                                                            curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices_MP)

                # All the things that should be revised:
            #     segment_branches, #skeleton branches
            #     divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
            #     segment_widths_median


                new_submeshes = [k["branch_mesh"] for k in local_correspondence_revised.values()]
                new_submeshes_idx = [extend_soma_mesh_idx[k["branch_face_idx"]] for k in local_correspondence_revised.values()]
                new_skeletal_branches = [k["branch_skeleton"] for k in local_correspondence_revised.values()]

                #calculate the new width
                ray_inter = tu.ray_pyembree.RayMeshIntersector(limb_mesh_mparty)
                new_widths = []
                for new_s_idx in new_submeshes_idx:
                    curr_ray_distance = tu.ray_trace_distance(mesh=limb_mesh_mparty, 
                                        face_inds=new_s_idx,
                                       ray_inter=ray_inter)
                    curr_width_median = np.median(curr_ray_distance[curr_ray_distance!=0])
                    print(f"curr_width_median = {curr_width_median}")
                    if (not np.isnan(curr_width_median)) and (curr_width_median > 0):
                        new_widths.append(curr_width_median)
                    else:
                        print(f"USING A DEFAULT WIDTH BECAUSE THE NEWLY COMPUTED ONE WAS {curr_width_median}: {segment_widths_median[match_sk_branches[0]]}")
                        new_widths.append(segment_widths_median[match_sk_branches[0]])


                #6) Remove the original branch and mesh correspondence and replace with the multiples
#                     print(f"match_sk_branches BEFORE = {match_sk_branches}")
#                     print(f"segment_branches BEFORE = {segment_branches}")
#                     print(f"len(new_skeletal_branches) = {len(new_skeletal_branches)}")
#                     print(f"new_skeletal_branches BEFORE= {new_skeletal_branches}")


                #segment_branches = np.delete(segment_branches,match_sk_branches,axis=0)
                #segment_branches = np.append(segment_branches,new_skeletal_branches,axis=0)

                segment_branches = np.array([k for i,k in enumerate(segment_branches) if i not in match_sk_branches] + new_skeletal_branches)


                divided_submeshes = np.delete(divided_submeshes,match_sk_branches,axis=0)
                divided_submeshes = np.append(divided_submeshes,new_submeshes,axis=0)


                #divided_submeshes_idx = np.delete(divided_submeshes_idx,match_sk_branches,axis=0)
                #divided_submeshes_idx = np.append(divided_submeshes_idx,new_submeshes_idx,axis=0)
                divided_submeshes_idx = np.array([k for i,k in enumerate(divided_submeshes_idx) if i not in match_sk_branches] + new_submeshes_idx)

                segment_widths_median = np.delete(segment_widths_median,match_sk_branches,axis=0)
                segment_widths_median = np.append(segment_widths_median,new_widths,axis=0)

                try:
                    debug = False
                    if debug:
                        print(f"segment_branches.shape = {segment_branches.shape}")
                        print(f"segment_branches = {segment_branches}")
                        print(f"new_skeletal_branches = {new_skeletal_branches}")
                    sk.check_skeleton_connected_component(sk.stack_skeletons(segment_branches))
                except:
                    su.compressed_pickle(local_correspondence_revised,"local_correspondence_revised")
                print("checked segment branches after soma add on")
                return_find = sk.find_branch_skeleton_with_specific_coordinate(segment_branches,
                                             orig_vertex)



                """ ******************* END OF HOW CAN DO STITCHING ********************  """



        limb_to_endpoints_must_keep_list.append(endpts_total)
        limb_to_soma_touching_vertices_list.append(curr_soma_to_piece_touching_vertices_total)

        # ------------------- 11/9 addition ------------------- #

        if no_soma_extension_add:
            print("No soma extending branch was added for this sublimb even though it had a soma border (means they already existed)")

        if print_fusion_steps:
            print(f"MP (because soma touching verts) soma extension add: {time.time() - fusion_time }")
            fusion_time = time.time()

    #building the limb correspondence
    limb_correspondence_MP[sublimb_idx] = dict()

    for zz,b_sk in enumerate(segment_branches):
        limb_correspondence_MP[sublimb_idx][zz] = dict(
            branch_skeleton = b_sk,
            width_from_skeleton = segment_widths_median[zz],
            branch_mesh = divided_submeshes[zz],
            branch_face_idx = divided_submeshes_idx[zz]
            )



limb_correspondence_MP_saved = copy.deepcopy(limb_correspondence_MP)
limb_correspondence_MAP_saved = copy.deepcopy(limb_correspondence_MAP)

# ------------------------------------- Part C: Will make sure the correspondences can all be stitched together --------------- #



#     su.compressed_pickle(limb_correspondence_MAP,"limb_correspondence_MAP_before_stitch")
#     su.compressed_pickle(limb_correspondence_MP,"limb_correspondence_MP_before_stitch")


if check_correspondence_branches:
    sk.check_correspondence_branches_have_2_endpoints(limb_correspondence_MAP)
    sk.check_correspondence_branches_have_2_endpoints(limb_correspondence_MP)

#total_keep_endpoints = np.concatenate([list(v.values()) for v in limb_to_endpoints_must_keep_list]).reshape(-1,3)
total_keep_endpoints = []
for entry in limb_to_endpoints_must_keep_list:
    for k,v in entry.items():
        total_keep_endpoints.append(v)
total_keep_endpoints = np.vstack(total_keep_endpoints)
    
    

# Only want to perform this step if both MP and MAP pieces
if len(limb_correspondence_MAP)>0 and len(limb_correspondence_MP)>0:

    # -------------- Part 11: Getting Sublimb Mesh and Skeletons and Gets connectivitiy by Mesh -------#
    # -------------(filtering connections to only MP to MAP edges)--------------- #

    # ---- Doing the mesh connectivity ---------#
    sublimb_meshes_MP = []
    sublimb_skeletons_MP = []

    for sublimb_key,sublimb_v in limb_correspondence_MP.items():
        sublimb_meshes_MP.append(tu.combine_meshes([branch_v["branch_mesh"] for branch_v in sublimb_v.values()]))
        sublimb_skeletons_MP.append(sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in sublimb_v.values()]))



    sublimb_meshes_MAP = []
    sublimb_skeletons_MAP = []


    for sublimb_key,sublimb_v in limb_correspondence_MAP.items():
        sublimb_meshes_MAP.append(tu.combine_meshes([branch_v["branch_mesh"] for branch_v in sublimb_v.values()]))
        sublimb_skeletons_MAP.append(sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in sublimb_v.values()]))

    sublimb_skeletons_MP_saved = copy.deepcopy(sublimb_skeletons_MP)
    sublimb_skeletons_MAP_saved = copy.deepcopy(sublimb_skeletons_MAP)

    connectivity_type = "edges"
    for i in range(0,2):
        mesh_conn,mesh_conn_vertex_groups = tu.mesh_list_connectivity(meshes = sublimb_meshes_MP + sublimb_meshes_MAP,
                                            main_mesh = limb_mesh_mparty,
                                            connectivity=connectivity_type,
                                            min_common_vertices=1,
                                            return_vertex_connection_groups=True,
                                            return_largest_vertex_connection_group=True,
                                            print_flag = False)
        mesh_conn_old = copy.deepcopy(mesh_conn)



        #check that every MAP piece mapped to a MP piece
        mesh_conn_filt = []
        mesh_conn_vertex_groups_filt = []
        for j,(m1,m2) in enumerate(mesh_conn):
            if m1 < len(sublimb_meshes_MP) and m2 >=len(sublimb_meshes_MP):
                mesh_conn_filt.append([m1,m2])
                mesh_conn_vertex_groups_filt.append(mesh_conn_vertex_groups[j])
            else:
                print(f"Edge {(m1,m2)} was not kept")
        mesh_conn_filt = np.array(mesh_conn_filt)

        mesh_conn = mesh_conn_filt
        mesh_conn_vertex_groups = mesh_conn_vertex_groups_filt

        #check that the mapping should create only one connected component
        G = nx.from_edgelist(mesh_conn)



        try:
            if len(G) != len(sublimb_meshes_MP) + len(sublimb_meshes_MAP):
                raise Exception("Number of nodes in mesh connectivity graph is not equal to number of  MAP and MP sublimbs")

            connect_comp = list(nx.connected_components(G))
            if len(connect_comp)>1:
                raise Exception(f"Mesh connectivity was not one component, instead it was ({len(connect_comp)}): {connect_comp} ")
        except:

            if connectivity_type == "vertices":
                print(f"mesh_conn_filt = {mesh_conn_filt}")
                print(f"mesh_conn_old = {mesh_conn_old}")
                mesh_conn_adjusted = np.vstack([mesh_conn[:,0],mesh_conn[:,1]-len(sublimb_meshes_MP)]).T
                print(f"mesh_conn_adjusted = {mesh_conn_adjusted}")
                print(f"len(sublimb_meshes_MP) = {len(sublimb_meshes_MP)}")
                print(f"len(sublimb_meshes_MAP) = {len(sublimb_meshes_MAP)}")
                meshes = sublimb_meshes_MP + sublimb_meshes_MAP
                #su.compressed_pickle(meshes,"meshes")
                su.compressed_pickle(sublimb_meshes_MP,"sublimb_meshes_MP")
                su.compressed_pickle(sublimb_meshes_MAP,"sublimb_meshes_MAP")
                su.compressed_pickle(limb_mesh_mparty,"limb_mesh_mparty")
                su.compressed_pickle(sublimb_skeletons_MP,"sublimb_skeletons_MP")
                su.compressed_pickle(sublimb_skeletons_MAP,"sublimb_skeletons_MAP")




                raise Exception("Something went wrong in the connectivity")
            else:
                print(f"Failed on connection type {connectivity_type} ")
                connectivity_type = "vertices"
                print(f"so changing type to {connectivity_type}")
        else:
            print(f"Successful mesh connectivity with type {connectivity_type}")
            break


    #adjust the connection indices for MP and MAP indices
    mesh_conn_adjusted = np.vstack([mesh_conn[:,0],mesh_conn[:,1]-len(sublimb_meshes_MP)]).T






    """
    Pseudocode:
    For each connection edge:
        For each vertex connection group:
            1) Get the endpoint vertices of the MP skeleton
            2) Find the closest endpoint vertex to the vertex connection group (this is MP stitch point)
            3) Find the closest skeletal point on MAP pairing (MAP stitch) 
            4) Find the branches that have that MAP stitch point:
            5A) If the number of branches corresponding to stitch point is multipled
                --> then we are stitching at a branching oint
                i) Just add the skeletal segment from MP_stitch to MAP stitch to the MP skeletal segment
                ii) 

    """



    # -------------- STITCHING PHASE -------#
    stitch_counter = 0
    all_map_stitch_points = []
    for (MP_idx,MAP_idx),v_g in zip(mesh_conn_adjusted,mesh_conn_vertex_groups):
        print(f"\n---- Working on {(MP_idx,MAP_idx)} connection-----")

        """
        This old way of getting the endpoints was not good because could possibly just need
        a stitching done between original branch junction

        skeleton_MP_graph = sk.convert_skeleton_to_graph(curr_skeleton_MP)
        endpoint_nodes = xu.get_nodes_of_degree_k(skeleton_MP_graph,1)
        endpoint_nodes_coordinates = xu.get_node_attributes(skeleton_MP_graph,node_list=endpoint_nodes)
        """


        # -------------- Part 12: Find the MP and MAP stitching point and branches that contain the stitching point-------#

        """  OLD WAY THAT ALLOWED STITICHING POINTS TO NOT BE CONNECTED AT THE CONNECTING BRANCHES
        #getting the skeletons that should be stitched
        curr_skeleton_MP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MP[MP_idx].values()])
        curr_skeleton_MAP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MAP[MAP_idx].values()])

        #1) Get the endpoint vertices of the MP skeleton branches (so every endpoint or high degree node)
        #(needs to be inside loop because limb correspondence will change)
        curr_MP_branch_skeletons = [limb_correspondence_MP[MP_idx][k]["branch_skeleton"] for k in np.sort(list(limb_correspondence_MP[MP_idx].keys()))]
        endpoint_nodes_coordinates = np.array([sk.find_branch_endpoints(k) for k in curr_MP_branch_skeletons])
        endpoint_nodes_coordinates = np.unique(endpoint_nodes_coordinates.reshape(-1,3),axis=0)

        #2) Find the closest endpoint vertex to the vertex connection group (this is MP stitch point)
        av_vert = np.mean(v_g,axis=0)
        winning_vertex = endpoint_nodes_coordinates[np.argmin(np.linalg.norm(endpoint_nodes_coordinates-av_vert,axis=1))]
        print(f"winning_vertex = {winning_vertex}")


        #2b) Find the branch points where the winning vertex is located
        MP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
            divded_skeleton=curr_MP_branch_skeletons,
            current_coordinate = winning_vertex
        )
        print(f"MP_branches_with_stitch_point = {MP_branches_with_stitch_point}")


        #3) Find the closest skeletal point on MAP pairing (MAP stitch)
        MAP_skeleton_coords = np.unique(curr_skeleton_MAP.reshape(-1,3),axis=0)
        MAP_stitch_point = MAP_skeleton_coords[np.argmin(np.linalg.norm(MAP_skeleton_coords-winning_vertex,axis=1))]


        #3b) Consider if the stitch point is close enough to end or branch node in skeleton:
        # and if so then reassign
        if move_MAP_stitch_to_end_or_branch:
            MAP_stitch_point_new,change_status = sk.move_point_to_nearest_branch_end_point_within_threshold(
                                                    skeleton=curr_skeleton_MAP,
                                                    coordinate=MAP_stitch_point,
                                                    distance_to_move_point_threshold = distance_to_move_point_threshold,
                                                    verbose=True

                                                    )
            MAP_stitch_point=MAP_stitch_point_new


        #4) Find the branches that have that MAP stitch point:
        curr_MAP_branch_skeletons = [limb_correspondence_MAP[MAP_idx][k]["branch_skeleton"]
                                         for k in np.sort(list(limb_correspondence_MAP[MAP_idx].keys()))]

        MAP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
            divded_skeleton=curr_MAP_branch_skeletons,
            current_coordinate = MAP_stitch_point
        )



        MAP_stitch_point_on_end_or_branch = False
        if len(MAP_branches_with_stitch_point)>1:
            MAP_stitch_point_on_end_or_branch = True
        elif len(MAP_branches_with_stitch_point)==1:
            if len(nu.matching_rows(sk.find_branch_endpoints(curr_MAP_branch_skeletons[MAP_branches_with_stitch_point[0]]),
                                    MAP_stitch_point))>0:
                MAP_stitch_point_on_end_or_branch=True
        else:
            raise Exception("No matching MAP values")

    """

        #*****should only get branches that are touching....****

        #getting the skeletons that should be stitched
        curr_skeleton_MP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MP[MP_idx].values()])
        curr_skeleton_MAP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MAP[MAP_idx].values()])


        av_vert = np.mean(v_g,axis=0)

        # ---------------- Doing the MAP part first -------------- #
        """
        The previous way did not ensure that the MAP point found will have a branch mesh that is touching the border vertices

        #3) Find the closest skeletal point on MAP pairing (MAP stitch)
        MAP_skeleton_coords = np.unique(curr_skeleton_MAP.reshape(-1,3),axis=0)

        #this does not guarentee that the MAP branch associated with the MAP stitch point is touching the border group
        MAP_stitch_point = MAP_skeleton_coords[np.argmin(np.linalg.norm(MAP_skeleton_coords-av_vert,axis=1))]
        """

        # -------------- 11/9 NEW METHOD FOR FINDING MAP STITCH POINT ------------ #
        o_keys = np.sort(list(limb_correspondence_MAP[MAP_idx].keys()))
        curr_MAP_branch_meshes = np.array([limb_correspondence_MAP[MAP_idx][k]["branch_mesh"]
                                         for k in o_keys])
        curr_MAP_branch_skeletons = np.array([limb_correspondence_MAP[MAP_idx][k]["branch_skeleton"]
                                         for k in o_keys])

        MAP_pieces_idx_touching_border = tu.filter_meshes_by_containing_coordinates(mesh_list=curr_MAP_branch_meshes,
                                       nullifying_points=v_g,
                                        filter_away=False,
                                       distance_threshold=0,
                                       return_indices=True)

        MAP_branches_considered = curr_MAP_branch_skeletons[MAP_pieces_idx_touching_border]
        curr_skeleton_MAP_for_stitch = sk.stack_skeletons(MAP_branches_considered)

        #3) Find the closest skeletal point on MAP pairing (MAP stitch)
        MAP_skeleton_coords = np.unique(curr_skeleton_MAP_for_stitch.reshape(-1,3),axis=0)

        """
        #------- OLD WAY: this does not guarentee that the MAP branch associated with the MAP stitch point is touching the border group
        #MAP_stitch_point = MAP_skeleton_coords[np.argmin(np.linalg.norm(MAP_skeleton_coords-av_vert,axis=1))]
        
        # ------- 1/1/21 Change to make sure never stitches to soma connecting point ----
        Pseudocode: 
        1) Get all the closest coordinates and sort in order of distance
        2) Iterate through the top coordinates:
        - check if not in the endpoints
        a. if not --> make that the winning MAP stitch point
        b. if not --> continue to next

        3) if get to end and dont have winning coordinate then error
        """

        MAP_stitch_point = MAP_skeleton_coords[np.argmin(np.linalg.norm(MAP_skeleton_coords-av_vert,axis=1))]

        # --------- 11/13: Making so could possibly stitch to another point that was already stitched to
        curr_br_endpts = np.array([sk.find_branch_endpoints(k) for k in MAP_branches_considered]).reshape(-1,3)
        curr_br_endpts_unique = np.unique(curr_br_endpts,axis=0)
        



        #3b) Consider if the stitch point is close enough to end or branch node in skeleton:
        # and if so then reassign
        print(f"total_keep_endpoints = {total_keep_endpoints}")
        print(f"MAP_stitch_point BEFORE = {MAP_stitch_point}, move_MAP_stitch_to_end_or_branch = {move_MAP_stitch_to_end_or_branch}")
        
        if move_MAP_stitch_to_end_or_branch:
            MAP_stitch_point_new,change_status = sk.move_point_to_nearest_branch_end_point_within_threshold(
                                                    skeleton=curr_skeleton_MAP,
                                                    coordinate=MAP_stitch_point,
                                                    distance_to_move_point_threshold = distance_to_move_point_threshold,
                                                    verbose=True,
                                                    possible_node_coordinates=curr_br_endpts_unique,
                                                    excluded_node_coordinates=total_keep_endpoints,
                                                    )
            MAP_stitch_point=MAP_stitch_point_new
            
        print(f"MAP_stitch_point AFTER = {MAP_stitch_point}")
        
        if len(nu.matching_rows(total_keep_endpoints,MAP_stitch_point))>0:
            raise Exception("About to stitch to soma connecting point")

        #4) Find the branches that have that MAP stitch point:

        MAP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
            divded_skeleton=curr_MAP_branch_skeletons,
            current_coordinate = MAP_stitch_point
        )



        MAP_stitch_point_on_end_or_branch = False
        if len(MAP_branches_with_stitch_point)>1:
            MAP_stitch_point_on_end_or_branch = True
        elif len(MAP_branches_with_stitch_point)==1:
            if len(nu.matching_rows(sk.find_branch_endpoints(curr_MAP_branch_skeletons[MAP_branches_with_stitch_point[0]]),
                                    MAP_stitch_point))>0:
                MAP_stitch_point_on_end_or_branch=True
        else:
            raise Exception("No matching MAP values")

        #add the map stitch point to the history
        all_map_stitch_points.append(MAP_stitch_point)

        # ---------------- Doing the MP Part --------------------- #



        ord_keys = np.sort(list(limb_correspondence_MP[MP_idx].keys()))
        curr_MP_branch_meshes = [limb_correspondence_MP[MP_idx][k]["branch_mesh"] for k in ord_keys]



        """ old way of filtering MP pieces just to those touching the MAP, but just want the ones touching the connection group

        MAP_meshes_with_stitch_point = tu.combine_meshes([limb_correspondence_MAP[MAP_idx][k]["branch_mesh"] for k in MAP_branches_with_stitch_point])

        conn = tu.mesh_pieces_connectivity(main_mesh=limb_mesh_mparty,
                                   central_piece=MAP_meshes_with_stitch_point,
                                   periphery_pieces=curr_MP_branch_meshes)
        """
        # 11/9 Addition: New way that filters meshes by their touching of the vertex connection group (this could possibly be an empty group)
        conn = tu.filter_meshes_by_containing_coordinates(mesh_list=curr_MP_branch_meshes,
                                       nullifying_points=v_g,
                                        filter_away=False,
                                       distance_threshold=0,
                                       return_indices=True)

        if len(conn) == 0:
            print("Connectivity was 0 for the MP mesh groups touching the vertex group so not restricting by that anymore")
            sk_conn = np.arange(0,len(curr_MP_branch_meshes))
        else:
            sk_conn = conn

        print(f"sk_conn = {sk_conn}")
        print(f"conn = {conn}")


        #1) Get the endpoint vertices of the MP skeleton branches (so every endpoint or high degree node)
        #(needs to be inside loop because limb correspondence will change)
        curr_MP_branch_skeletons = [limb_correspondence_MP[MP_idx][k]["branch_skeleton"] for k in sk_conn]
        endpoint_nodes_coordinates = np.array([sk.find_branch_endpoints(k) for k in curr_MP_branch_skeletons])
        endpoint_nodes_coordinates = np.unique(endpoint_nodes_coordinates.reshape(-1,3),axis=0)
        
        """ ---------- 1 /5: Take out the possible endpoints --------------------"""
        if prevent_MP_starter_branch_stitches:
            endpoint_nodes_coordinates = nu.setdiff2d(endpoint_nodes_coordinates,total_keep_endpoints)


        #2) Find the closest endpoint vertex to the vertex connection group (this is MP stitch point)

        winning_vertex = endpoint_nodes_coordinates[np.argmin(np.linalg.norm(endpoint_nodes_coordinates-av_vert,axis=1))]
        print(f"winning_vertex = {winning_vertex}")
        
        if len(nu.matching_rows(total_keep_endpoints,winning_vertex))>0:
            raise Exception("Trying to stitch to MP starting point")
        
        


        #2b) Find the branch points where the winning vertex is located
        curr_MP_branch_skeletons = [limb_correspondence_MP[MP_idx][k]["branch_skeleton"] for k in np.sort(list(limb_correspondence_MP[MP_idx].keys()))]
        MP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
            divded_skeleton=curr_MP_branch_skeletons,
            current_coordinate = winning_vertex
        )
        
        print(f"MP_branches_with_stitch_point = {MP_branches_with_stitch_point}")



        print(f"MAP_branches_with_stitch_point = {MAP_branches_with_stitch_point}")
        print(f"MAP_stitch_point_on_end_or_branch = {MAP_stitch_point_on_end_or_branch}")


        # -------- 11/13 addition: Will see if the MP stitch point was already a MAP stitch point ---- #
        if len(nu.matching_rows(np.array(all_map_stitch_points),winning_vertex)) > 0:
            keep_MP_stitch_static = True
        else:
            keep_MP_stitch_static = False





        # ------------------------- This part does the stitching -------------------- #


        """
        Pseudocode:
        1) For all MP branches
            a) Get neighbor coordinates to MP stitch points
            b) Delete the MP Stitch points on each 
            c) Add skeleton segment from neighbor to MAP stitch point
        2) Get skeletons and meshes from MP and MAP pieces
        3) Run mesh correspondence to get new meshes and mesh_idx and widths
        4a) If MAP_stitch_point_on_end_or_branch is False
        - Delete the old MAP branch parts and replace with new MAP ones
        4b) Revise the meshes,  mesh_idx, and widths of the MAP pieces
        5) Revise the meshes,  mesh_idx, and widths of the MP pieces


        """

        # -------------- Part 13: Will Adjust the MP branches that have the stitch point so extends to the MAP stitch point -------#
        curr_MP_sk = []
        for b_idx in MP_branches_with_stitch_point:
            if not keep_MP_stitch_static:
                #a) Get neighbor coordinates to MP stitch points
                MP_stitch_branch_graph = sk.convert_skeleton_to_graph(curr_MP_branch_skeletons[b_idx])
                stitch_node = xu.get_nodes_with_attributes_dict(MP_stitch_branch_graph,dict(coordinates=winning_vertex))[0]
                stitch_neighbors = xu.get_neighbors(MP_stitch_branch_graph,stitch_node)

                if len(stitch_neighbors) != 1:
                    raise Exception("Not just one neighbor for stitch point of MP branch")
                keep_neighbor = stitch_neighbors[0]  
                keep_neighbor_coordinates = xu.get_node_attributes(MP_stitch_branch_graph,node_list=[keep_neighbor])[0]

                #b) Delete the MP Stitch points on each 
                MP_stitch_branch_graph.remove_node(stitch_node)

                """ Old way that does not do smoothing

                #c) Add skeleton segment from neighbor to MAP stitch point
                new_node_name = np.max(MP_stitch_branch_graph.nodes())+1

                MP_stitch_branch_graph.add_nodes_from([(int(new_node_name),{"coordinates":MAP_stitch_point})])
                MP_stitch_branch_graph.add_weighted_edges_from([(keep_neighbor,new_node_name,np.linalg.norm(MAP_stitch_point - keep_neighbor_coordinates))])

                new_MP_skeleton = sk.convert_graph_to_skeleton(MP_stitch_branch_graph)

                """
                try:
                    if len(MP_stitch_branch_graph)>1:
                        new_MP_skeleton = sk.add_and_smooth_segment_to_branch(skeleton=sk.convert_graph_to_skeleton(MP_stitch_branch_graph),
                                                        skeleton_stitch_point=keep_neighbor_coordinates,
                                                         new_stitch_point=MAP_stitch_point)
                    else:
                        print("Not even attempting smoothing segment because once keep_neighbor_coordinates")
                        new_MP_skeleton = np.vstack([keep_neighbor_coordinates,MAP_stitch_point]).reshape(-1,2,3)
                except:
                    su.compressed_pickle(MP_stitch_branch_graph,"MP_stitch_branch_graph")
                    su.compressed_pickle(keep_neighbor_coordinates,"keep_neighbor_coordinates")
                    su.compressed_pickle(MAP_stitch_point,"MAP_stitch_point")


                    raise Exception("Something went wrong with add_and_smooth_segment_to_branch")





                #smooth over the new skeleton
                new_MP_skeleton_smooth = sk.resize_skeleton_branch(new_MP_skeleton,
                                                                  segment_width=meshparty_segment_size)

                curr_MP_sk.append(new_MP_skeleton_smooth)
            else:
                print(f"Not adjusting MP skeletons because keep_MP_stitch_static = {keep_MP_stitch_static}")
                curr_MP_sk.append(curr_MP_branch_skeletons[b_idx])



        #2) Get skeletons and meshes from MP and MAP pieces
        curr_MAP_sk = [limb_correspondence_MAP[MAP_idx][k]["branch_skeleton"] for k in MAP_branches_with_stitch_point]

        #2.1) Going to break up the MAP skeleton if need be
        """
        Pseudocode:
        a) check to see if it needs to be broken up
        If it does:
        b) Convert the skeleton into a graph
        c) Find the node of the MAP stitch point (where need to do the breaking)
        d) Find the degree one nodes
        e) For each degree one node:
        - Find shortest path from stitch node to end node
        - get a subgraph from that path
        - convert graph to a skeleton and save as new skeletons

        """
        # -------------- Part 14: Breaks Up MAP skeleton into 2 pieces if Needs (because MAP stitch point not on endpoint or branch point)  -------#

        #a) check to see if it needs to be broken up
        cut_flag = False
        if not MAP_stitch_point_on_end_or_branch:
            if len(curr_MAP_sk) > 1:
                raise Exception(f"There was more than one skeleton for MAP skeletons even though MAP_stitch_point_on_end_or_branch = {MAP_stitch_point_on_end_or_branch}")


            skeleton_to_cut = curr_MAP_sk[0]
            curr_MAP_sk = sk.cut_skeleton_at_coordinate(skeleton=skeleton_to_cut,
                                                        cut_coordinate=MAP_stitch_point)
            cut_flag=True


        # ------ 11/13 Addition: need to adjust the MAP points if have to keep MP static
        if keep_MP_stitch_static:
            curr_MAP_sk_final = []
            for map_skel in curr_MAP_sk:
                #a) Get neighbor coordinates to MP stitch points
                MP_stitch_branch_graph = sk.convert_skeleton_to_graph(map_skel)
                stitch_node = xu.get_nodes_with_attributes_dict(MP_stitch_branch_graph,dict(coordinates=MAP_stitch_point))[0]
                stitch_neighbors = xu.get_neighbors(MP_stitch_branch_graph,stitch_node)

                if len(stitch_neighbors) != 1:
                    raise Exception("Not just one neighbor for stitch point of MP branch")
                keep_neighbor = stitch_neighbors[0]  
                keep_neighbor_coordinates = xu.get_node_attributes(MP_stitch_branch_graph,node_list=[keep_neighbor])[0]

                #b) Delete the MP Stitch points on each 
                MP_stitch_branch_graph.remove_node(stitch_node)

                """ Old way that does not do smoothing

                #c) Add skeleton segment from neighbor to MAP stitch point
                new_node_name = np.max(MP_stitch_branch_graph.nodes())+1

                MP_stitch_branch_graph.add_nodes_from([(int(new_node_name),{"coordinates":MAP_stitch_point})])
                MP_stitch_branch_graph.add_weighted_edges_from([(keep_neighbor,new_node_name,np.linalg.norm(MAP_stitch_point - keep_neighbor_coordinates))])

                new_MP_skeleton = sk.convert_graph_to_skeleton(MP_stitch_branch_graph)

                """
                try:
                    if len(MP_stitch_branch_graph)>1:
                        new_MP_skeleton = sk.add_and_smooth_segment_to_branch(skeleton=sk.convert_graph_to_skeleton(MP_stitch_branch_graph),
                                                        skeleton_stitch_point=keep_neighbor_coordinates,
                                                         new_stitch_point=winning_vertex)
                    else:
                        print("Not even attempting smoothing segment because once keep_neighbor_coordinates")
                        new_MP_skeleton = np.vstack([keep_neighbor_coordinates,MAP_stitch_point]).reshape(-1,2,3)
                except:
                    su.compressed_pickle(MP_stitch_branch_graph,"MP_stitch_branch_graph")
                    su.compressed_pickle(keep_neighbor_coordinates,"keep_neighbor_coordinates")
                    su.compressed_pickle(winning_vertex,"winning_vertex")


                    raise Exception("Something went wrong with add_and_smooth_segment_to_branch")





                #smooth over the new skeleton
                new_MP_skeleton_smooth = sk.resize_skeleton_branch(new_MP_skeleton,
                                                                  segment_width=meshparty_segment_size)

                curr_MAP_sk_final.append(new_MP_skeleton_smooth)
            curr_MAP_sk = copy.deepcopy(curr_MAP_sk_final)



        # -------------- Part 15: Gets all of the skeletons and Mesh to divide u and does mesh correspondence -------#
        # ------------- revise IDX so still references the whole limb mesh -----------#

        # -------------- 11/10 Addition accounting for not all MAP pieces always touching each other --------------------#
        if len(MAP_branches_with_stitch_point) > 1:
            print("\nRevising the MAP pieces index:")
            print(f"MAP_pieces_idx_touching_border = {MAP_pieces_idx_touching_border}, MAP_branches_with_stitch_point = {MAP_branches_with_stitch_point}")
            MAP_pieces_for_correspondence = nu.intersect1d(MAP_pieces_idx_touching_border,MAP_branches_with_stitch_point)
            print(f"MAP_pieces_for_correspondence = {MAP_pieces_for_correspondence}")
            curr_MAP_sk = [limb_correspondence_MAP[MAP_idx][k]["branch_skeleton"] for k in MAP_pieces_for_correspondence]
        else:
            MAP_pieces_for_correspondence = MAP_branches_with_stitch_point

        curr_MAP_meshes_idx = [limb_correspondence_MAP[MAP_idx][k]["branch_face_idx"] for k in MAP_pieces_for_correspondence]

        # Have to adjust based on if the skeleton were split

        if cut_flag:
            #Then it was cut and have to do mesh correspondence to find what label to cut
            if len(curr_MAP_meshes_idx) > 1:
                raise Exception("MAP_pieces_for_correspondence was longer than 1 and cut flag was set")
            pre_stitch_mesh_idx = curr_MAP_meshes_idx[0]
            pre_stitch_mesh = limb_mesh_mparty.submesh([pre_stitch_mesh_idx],append=True,repair=False)
            local_correspondnece_stitch = mesh_correspondence_first_pass(mesh=pre_stitch_mesh,
                                      skeleton_branches=curr_MAP_sk)
            local_correspondence_stitch_revised_MAP = correspondence_1_to_1(mesh=pre_stitch_mesh,
                                                        local_correspondence=local_correspondnece_stitch,
                                                        curr_limb_endpoints_must_keep=None,
                                                        curr_soma_to_piece_touching_vertices=None)

#                 curr_MAP_meshes_idx = [pre_stitch_mesh_idx[local_correspondence_stitch_revised_MAP[nn]["branch_face_idx"]] for 
#                                                nn in local_correspondence_stitch_revised_MAP.keys()]

            #Need to readjust the mesh correspondence idx
            for k,v in local_correspondence_stitch_revised_MAP.items():
                local_correspondence_stitch_revised_MAP[k]["branch_face_idx"] = pre_stitch_mesh_idx[local_correspondence_stitch_revised_MAP[k]["branch_face_idx"]]

            curr_MAP_meshes_idx = [v["branch_face_idx"] for v in local_correspondence_stitch_revised_MAP.values()]
        else:
            local_correspondence_stitch_revised_MAP = dict([(gg,limb_correspondence_MAP[MAP_idx][kk]) for gg,kk in enumerate(MAP_pieces_for_correspondence)])

            for gg,kk in enumerate(MAP_pieces_for_correspondence):
                local_correspondence_stitch_revised_MAP[gg]["branch_skeleton"] = curr_MAP_sk[gg]



        #To make sure that the MAP never gives up ground on the labels
        must_keep_labels_MAP = dict()
        must_keep_counter = 0
        for kk,b_idx in enumerate(curr_MAP_meshes_idx):
            #must_keep_labels_MAP.update(dict([(ii,kk) for ii in range(must_keep_counter,must_keep_counter+len(b_idx))]))
            must_keep_labels_MAP[kk] = np.arange(must_keep_counter,must_keep_counter+len(b_idx))
            must_keep_counter += len(b_idx)



        #this is where should send only the MP that apply
        MP_branches_for_correspondence,conn_idx,MP_branches_with_stitch_point_idx = nu.intersect1d(conn,MP_branches_with_stitch_point,return_indices=True)

        curr_MP_meshes_idx = [limb_correspondence_MP[MP_idx][k]["branch_face_idx"] for k in MP_branches_for_correspondence]
        curr_MP_sk_for_correspondence = [curr_MP_sk[zz] for zz in MP_branches_with_stitch_point_idx]

        stitching_mesh_idx = np.concatenate(curr_MAP_meshes_idx + curr_MP_meshes_idx)
        stitching_mesh = limb_mesh_mparty.submesh([stitching_mesh_idx],append=True,repair=False)
        stitching_skeleton_branches = curr_MAP_sk + curr_MP_sk_for_correspondence
        """

        ****** NEED TO GET THE RIGHT MESH TO RUN HE IDX ON SO GETS A GOOD MESH (CAN'T BE LIMB_MESH_MPARTY)
        BUT MUST BE THE ORIGINAL MAP MESH

        mesh_pieces_for_MAP
        sublimb_meshes_MP

        mesh_pieces_for_MAP_face_idx
        sublimb_meshes_MP_face_idx

        stitching_mesh = tu.combine_meshes(curr_MAP_meshes + curr_MP_meshes)
        stitching_skeleton_branches = curr_MAP_sk + curr_MP_sk

        """

        # ******************************** this is where should do thing about no mesh correspondence ***************** #

        # -------- 12/22: Trying to do the re-correspondence but if doesn't work then just resort to old one --------- #

        try:

            #3) Run mesh correspondence to get new meshes and mesh_idx and widths
            local_correspondnece_stitch = mesh_correspondence_first_pass(mesh=stitching_mesh,
                                          skeleton_branches=stitching_skeleton_branches)

            local_correspondence_stitch_revised = correspondence_1_to_1(mesh=stitching_mesh,
                                                        local_correspondence=local_correspondnece_stitch,
                                                        curr_limb_endpoints_must_keep=None,
                                                        curr_soma_to_piece_touching_vertices=None,
                                                        must_keep_labels=must_keep_labels_MAP)

            #Need to readjust the mesh correspondence idx
            for k,v in local_correspondence_stitch_revised.items():
                local_correspondence_stitch_revised[k]["branch_face_idx"] = stitching_mesh_idx[local_correspondence_stitch_revised[k]["branch_face_idx"]]
        except:
            print("Errored in 1 to 1 correspondence in stitching so just reverting to the original mesh assignments")
            # Setting the correspondence manually because the adaptive way did not work
            local_counter = 0
            local_correspondence_stitch_revised = dict()

            # setting the MAP parts (the new skeletons have already been adjusted)
            for k in local_correspondence_stitch_revised_MAP:
                local_correspondence_stitch_revised[local_counter] = local_correspondence_stitch_revised_MAP[k]
                local_counter += 1

            # setting the MP parts (the new skeletons have not been adjusted yet so adjusting them here)
            for mp_idx, k in enumerate(MP_branches_for_correspondence):
                local_correspondence_stitch_revised[local_counter] = limb_correspondence_MP[MP_idx][k] 
                local_correspondence_stitch_revised[local_counter]["branch_skeleton"] = curr_MP_sk[mp_idx]
                local_counter += 1


#                 su.compressed_pickle(stitching_skeleton_branches,"stitching_skeleton_branches")
#                 su.compressed_pickle(stitching_mesh,"stitching_mesh")
#                 su.compressed_pickle(local_correspondnece_stitch,"local_correspondnece_stitch")
#                 su.compressed_pickle(must_keep_labels_MAP,"must_keep_labels_MAP")

#                 raise Exception("Something went wrong with 1 to 1 correspondence")







        # -------------- Part 16: Overwrite old branch entries (and add on one new to MAP if required a split) -------#


        #4a) If MAP_stitch_point_on_end_or_branch is False
        #- Delete the old MAP branch parts and replace with new MAP ones
        if not MAP_stitch_point_on_end_or_branch:
            print("Deleting branches from dictionary")
            del limb_correspondence_MAP[MAP_idx][MAP_branches_with_stitch_point[0]]
            #adding the two new branches created from the stitching
            limb_correspondence_MAP[MAP_idx][MAP_branches_with_stitch_point[0]] = local_correspondence_stitch_revised[0]
            limb_correspondence_MAP[MAP_idx][np.max(list(limb_correspondence_MAP[MAP_idx].keys()))+1] = local_correspondence_stitch_revised[1]

            #have to reorder the keys
            #limb_correspondence_MAP[MAP_idx] = dict([(k,limb_correspondence_MAP[MAP_idx][k]) for k in np.sort(list(limb_correspondence_MAP[MAP_idx].keys()))])
            limb_correspondence_MAP[MAP_idx] = gu.order_dict_by_keys(limb_correspondence_MAP[MAP_idx])

        else: #4b) Revise the meshes,  mesh_idx, and widths of the MAP pieces if weren't broken up
            for j,curr_MAP_idx_fixed in enumerate(MAP_pieces_for_correspondence): 
                limb_correspondence_MAP[MAP_idx][curr_MAP_idx_fixed] = local_correspondence_stitch_revised[j]
            #want to update all of the skeletons just in case was altered by keep_MP_stitch_static and not included in correspondence
            if keep_MP_stitch_static:
                if len(MAP_branches_with_stitch_point) != len(curr_MAP_sk_final):
                    raise Exception("MAP_branches_with_stitch_point not same size as curr_MAP_sk_final")
                for gg,map_idx_curr in enumerate(MAP_branches_with_stitch_point):
                    limb_correspondence_MAP[MAP_idx][map_idx_curr]["branch_skeleton"] = curr_MAP_sk_final[gg]


        for j,curr_MP_idx_fixed in enumerate(MP_branches_for_correspondence): #************** right here just need to make only the ones that applied
            limb_correspondence_MP[MP_idx][curr_MP_idx_fixed] = local_correspondence_stitch_revised[j+len(curr_MAP_sk)]


        #5b) Fixing the branch skeletons that were not included in the correspondence
        MP_leftover,MP_leftover_idx = nu.setdiff1d(MP_branches_with_stitch_point,MP_branches_for_correspondence)
        print(f"MP_branches_with_stitch_point= {MP_branches_with_stitch_point}")
        print(f"MP_branches_for_correspondence = {MP_branches_for_correspondence}")
        print(f"MP_leftover = {MP_leftover}, MP_leftover_idx = {MP_leftover_idx}")

        for curr_MP_leftover,curr_MP_leftover_idx in zip(MP_leftover,MP_leftover_idx):
            limb_correspondence_MP[MP_idx][curr_MP_leftover]["branch_skeleton"] = curr_MP_sk[curr_MP_leftover_idx]


        print(f" Finished with {(MP_idx,MAP_idx)} \n\n\n")
        stitch_counter += 1
#         if cut_flag:
#             raise Exception("Cut flag was activated")

        if check_correspondence_branches:
            sk.check_correspondence_branches_have_2_endpoints(limb_correspondence_MAP[MAP_idx])
            sk.check_correspondence_branches_have_2_endpoints(limb_correspondence_MP[MP_idx])

#             su.compressed_pickle(limb_correspondence_MAP,f"limb_correspondence_MAP_{MAP_idx}_{MP_idx}")
#             su.compressed_pickle(limb_correspondence_MP,f"limb_correspondence_MP_{MAP_idx}_{MP_idx}")


else:
    print("There were not both MAP and MP pieces so skipping the stitch resolving phase")

print(f"Time for decomp of Limb = {time.time() - curr_limb_time}")
#     # ------------- Saving the MAP and MP Decompositions ---------------- #
#     proper_limb_mesh_correspondence_MAP[curr_limb_idx] = limb_correspondence_MAP
#     proper_limb_mesh_correspondence_MP[curr_limb_idx] = limb_correspondence_MP






# -------------- Part 17: Grouping the MP and MAP Correspondence into one correspondence dictionary -------#
limb_correspondence_individual = dict()
counter = 0

for sublimb_idx,sublimb_branches in limb_correspondence_MAP.items():
    for branch_dict in sublimb_branches.values():
        limb_correspondence_individual[counter]= branch_dict
        counter += 1
for sublimb_idx,sublimb_branches in limb_correspondence_MP.items():
    for branch_dict in sublimb_branches.values():
        limb_correspondence_individual[counter]= branch_dict
        counter += 1


#info that may be used for concept networks
network_starting_info = dict(
            touching_verts_list = limb_to_soma_touching_vertices_list,
            endpoints_must_keep = limb_to_endpoints_must_keep_list
)






# -------------- Part 18: 11-17 Addition that filters the network starting info into a more clean presentation ------------ #
"""
Pseudocode: 
1) Rearrange the network starting info into a ditionary mapping
  soma_idx --> branch_broder_group --> list of dict(touching_vertices,endpoint)

2) iterate through all the somas and border vertex groups
a. filter to only those with an endpoint that is on a branch of the skeleton
b1: If 1 --> then keep that one
b2: If more --> pick the one with the endpoint closest to the average fo the vertex group
b3: If 0 --> find the best available soma extending branch endpoint

"""

# Part 1: Rearrange network info


t_verts_list_total,enpts_list_total = network_starting_info.values()
network_starting_info_revised = dict()
for j,(v_list_dict,enpts_list_dict) in enumerate(zip(t_verts_list_total,enpts_list_total)):
    #print(f"---- Working on {j} -----")
#     print(v_list_dict)
#     print(enpts_list_dict)
    if set(list(v_list_dict.keys())) != set(list(enpts_list_dict)):
        raise Exception("Soma keys not match for touching vertices and endpoints")
    for sm_idx in v_list_dict.keys():
        v_list_soma = v_list_dict[sm_idx]
        endpt_soma = enpts_list_dict[sm_idx]
        if len(v_list_soma) != len(endpt_soma):
            raise Exception(f"touching vertices list and endpoint list not match size for soma {sm_idx}")

        all_border_vertex_groups = soma_touching_vertices_dict[sm_idx]

        for v_l,endpt in zip(v_list_soma,endpt_soma):

            matching_border_group  = []
            for i,curr_border_group in enumerate(all_border_vertex_groups):
                if nu.test_matching_vertices_in_lists(curr_border_group,v_l,verbose=True):
                    matching_border_group.append(i)

            if len(matching_border_group) == 0 or len(matching_border_group)>1:
                raise Exception(f"Matching border groups was not exactly 1: {matching_border_group}")

            winning_border_group = matching_border_group[0]

            if sm_idx not in network_starting_info_revised.keys():
                network_starting_info_revised[sm_idx] = dict()

            if winning_border_group not in network_starting_info_revised[sm_idx].keys():
                network_starting_info_revised[sm_idx][winning_border_group] = []
            network_starting_info_revised[sm_idx][winning_border_group].append(dict(touching_verts=v_l,endpoint=endpt))


# Part 2 Filter
"""
2) iterate through all the somas and border vertex groups
a. filter to only those with an endpoint that is on a branch of the skeleton
b1: If 1 --> then keep that one
b2: If more --> pick the one with the endpoint closest to the average fo the vertex group
b3: If 0 --> find the best available soma extending branch endpoint

Pseudocode for b3:
i) get all meshes that touch the vertex group (and keep the vertices that overlap)
--> error if none
ii) Get all of the endpoints of all matching branches
iii) Filter the endpoints to only those that are degree 1 in the overall skeleton
--> if none then just keep all endpoints
iv) Find the closest viable endpoint to the mean of the boundary group
v) save the overlap vertices and the winning endpoint as a dictionary

"""

limb_correspondence_individual_saved = copy.deepcopy(limb_correspondence_individual)

sorted_keys = np.sort(list(limb_correspondence_individual.keys()))
curr_branches = [limb_correspondence_individual[k]["branch_skeleton"] for k in sorted_keys]
curr_meshes = [limb_correspondence_individual[k]["branch_mesh"] for k in sorted_keys]

network_starting_info_revised_cleaned = dict()
for soma_idx in network_starting_info_revised.keys():
    network_starting_info_revised_cleaned[soma_idx] = dict()
    for bound_g_idx,endpoint_list in network_starting_info_revised[soma_idx].items():
        endpoint_list = np.array(endpoint_list)

        filter_on_skeleton_list = []
        for zz,endpt_dict in enumerate(endpoint_list):
            #a. filter to only those with an endpoint that is on a branch of the skeleton
            sk_indices = sk.find_branch_skeleton_with_specific_coordinate(divded_skeleton=curr_branches,
                                                                        current_coordinate=endpt_dict["endpoint"])
            if len(sk_indices) > 0:
                filter_on_skeleton_list.append(zz)

        endpoint_list_filt = endpoint_list[filter_on_skeleton_list]



        curr_border_group_coordinates = soma_touching_vertices_dict[soma_idx][bound_g_idx]
        boundary_mean = np.mean(curr_border_group_coordinates,axis=0)

        if len(endpoint_list_filt) == 1:
            print("Only one endpoint after filtering away the endpoints that are not on the skeleton")
            winning_dict = endpoint_list_filt[0]
        #b2: If more --> pick the one with the endpoint closest to the average fo the vertex group
        elif len(endpoint_list_filt) > 1:
            print(f"MORE THAN one endpoint after filtering away the endpoints that are not on the skeleton: {len(endpoint_list_filt)}")
            viable_endpoints = [endpt_dict["endpoint"] for endpt_dict in endpoint_list_filt]


            distanes_from_mean = np.linalg.norm(viable_endpoints-boundary_mean,axis=1)
            winning_endpoint_idx = np.argmin(distanes_from_mean)
            winning_dict = endpoint_list_filt[winning_endpoint_idx]

        #if there was no clear winner
        else:
            """
            Pseudocode for no viable options:
            i) get all meshes that touch the vertex group (and keep the vertices that overlap)
            --> error if none
            ii) Get all of the endpoints of all matching branches
            iii) Filter the endpoints to only those that are degree 1 in the overall skeleton
            --> if none then just keep all endpoints
            iv) Find the closest viable endpoint to the mean of the boundary group
            v) save the overlap vertices and the winning endpoint as a dictionary


            """
            print("Having to find a new branch point")
            #i) get all meshes that touch the vertex group (and keep the vertices that overlap)
            mesh_indices_on_border = tu.filter_meshes_by_containing_coordinates(curr_meshes,
                                          nullifying_points=curr_border_group_coordinates,
                                          filter_away=False,
                                          distance_threshold=0,
                                          return_indices=True)
            if len(mesh_indices_on_border) == 0:
                raise Exception("There were no meshes that were touching the boundary group")

            total_skeleton_graph = sk.convert_skeleton_to_graph(sk.stack_skeletons(curr_branches))
            skeleton_branches_on_border = [k for n,k in enumerate(curr_branches) if n in mesh_indices_on_border]
            skeleton_branches_on_border_endpoints = np.array([sk.find_branch_endpoints(k) for k in skeleton_branches_on_border])



            viable_endpoints = []
            for enpt in skeleton_branches_on_border_endpoints.reshape(-1,3):
                curr_enpt_node = xu.get_graph_node_by_coordinate(total_skeleton_graph,enpt,return_single_value=True)
                curr_enpt_degree = xu.get_node_degree(total_skeleton_graph,curr_enpt_node)
                #print(f"curr_enpt_degree = {curr_enpt_degree}")
                if curr_enpt_degree == 1:
                    viable_endpoints.append(enpt)

            if len(viable_endpoints) == 0:
                print("No branch endpoints were degree 1 so just using all endpoints")
                viable_endpoints = skeleton_branches_on_border_endpoints.reshape(-1,3)

            distanes_from_mean = np.linalg.norm(viable_endpoints-boundary_mean,axis=1)
            winning_endpoint = viable_endpoints[np.argmin(distanes_from_mean)]


            sk_indices = sk.find_branch_skeleton_with_specific_coordinate(divded_skeleton=curr_branches,
                                                                                    current_coordinate=winning_endpoint)

            winning_branch = np.intersect1d(mesh_indices_on_border,sk_indices)
            if len(winning_branch) == 0:
                raise Exception("There was no winning branch for the creation of a new soma extending branch")
            else:
                winning_branch_single = winning_branch[0]


            winning_touching_vertices = tu.filter_vertices_by_mesh(curr_meshes[winning_branch_single],curr_border_group_coordinates)
            winning_dict = dict(touching_verts=winning_touching_vertices,endpoint=winning_endpoint)








        network_starting_info_revised_cleaned[soma_idx][bound_g_idx] = winning_dict


# -------------- Part 18: Filter the limb correspondence for any short stubs ------------ #
if filter_end_nodes_from_correspondence:
    limb_correspondence_individual = pre.filter_limb_correspondence_for_end_nodes(limb_correspondence=limb_correspondence_individual,
                                                 mesh=limb_mesh_mparty,
                                                 starting_info=network_starting_info_revised_cleaned,
                                                filter_end_node_length=filter_end_node_length

                                                )





if not return_concept_network:
    if return_concept_network_starting_info: #because may want to calculate the concept networks later
        return_value = limb_correspondence_individual,network_starting_info_revised_cleaned
    else:
        return_value = limb_correspondence_individual
    raise Exception("Returned")
    
else:
    limb_to_soma_concept_networks = calculate_limb_concept_networks(limb_correspondence_individual,
                                                                    network_starting_info_revised_cleaned,
                                                                    run_concept_network_checks=run_concept_network_checks,
                                                                    error_on_starting_coordinates_not_endnodes= prevent_MP_starter_branch_stitches
                                                                   )




return_value =  limb_correspondence_individual,limb_to_soma_concept_networks
raise Exception("Returned")



In [ ]:
nviz.plot_limb_correspondence(limb_correspondence_individual)

# Debugging the missing mesh piece

In [ ]:
retrieved_obj = (minnie.Decomposition & dict(segment_id=segment_id)).fetch1("decomposition")

In [ ]:
debug_mesh = mesh_pieces_for_MAP[0]

In [ ]:
nviz.plot_objects(mesh_pieces_for_MAP[0])

In [ ]:
sk = reload(sk)

In [ ]:
for sublimb_idx,(mesh,mesh_idx) in enumerate(zip(mesh_pieces_for_MAP,mesh_pieces_for_MAP_face_idx)):
    print(f"--- Working on MAP piece {sublimb_idx}---")
    #print(f"soma_touching_vertices_dict = {soma_touching_vertices_dict}")
    mesh_start_time = time.time()
    curr_soma_to_piece_touching_vertices = filter_soma_touching_vertices_dict_by_mesh(
    mesh = mesh,
    curr_piece_to_soma_touching_vertices = soma_touching_vertices_dict
    )

    if print_fusion_steps:
        print(f"MAP Filtering Soma Pieces: {time.time() - fusion_time }")
        fusion_time = time.time()

    # ---- 0) Generating the Clean skeletons  -------------------------------------------#
    if not curr_soma_to_piece_touching_vertices is None:
        curr_total_border_vertices = dict([(k,np.vstack(v)) for k,v in curr_soma_to_piece_touching_vertices.items()])
    else:
        curr_total_border_vertices = None


    cleaned_branch,curr_limb_endpoints_must_keep = sk.skeletonize_and_clean_connected_branch_CGAL(
        mesh=mesh,
        curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices,
        total_border_vertices=curr_total_border_vertices,
        filter_end_node_length=filter_end_node_length,
        perform_cleaning_checks=perform_cleaning_checks,
        combine_close_skeleton_nodes = combine_close_skeleton_nodes,
        combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold,
    use_surface_after_CGAL=use_surface_after_CGAL,
    surface_reconstruction_size=surface_reconstruction_size,
    verbose=True)

    if not curr_limb_endpoints_must_keep is None:
        limb_to_endpoints_must_keep_list.append(curr_limb_endpoints_must_keep)
        limb_to_soma_touching_vertices_list.append(curr_soma_to_piece_touching_vertices)
    else:
        print("Inside MAP decomposition and curr_limb_endpoints_must_keep was None")

    if len(cleaned_branch) == 0:
        raise Exception(f"Found a zero length skeleton for limb {z} of trmesh {branch}")

    if print_fusion_steps:
        print(f"skeletonize_and_clean_connected_branch_CGAL: {time.time() - fusion_time }")
        fusion_time = time.time()

    # ---- 1) Generating Initial Mesh Correspondence -------------------------------------------#
    start_time = time.time()

    print(f"Working on limb correspondence for #{sublimb_idx} MAP piece")
    local_correspondence = mesh_correspondence_first_pass(mesh=mesh,
                                                         skeleton=cleaned_branch,
                                                         distance_by_mesh_center=distance_by_mesh_center,
                                                         connectivity="edges",
                                                         remove_inside_pieces_threshold=100)


    print(f"Total time for decomposition = {time.time() - start_time}")
    if print_fusion_steps:
        print(f"mesh_correspondence_first_pass: {time.time() - fusion_time }")
        fusion_time = time.time()


    #------------- 2) Doing Some checks on the initial corespondence -------- #


    if perform_cleaning_checks:
        check_skeletonization_and_decomp(skeleton=cleaned_branch,
                                        local_correspondence=local_correspondence)

    # -------3) Finishing off the face correspondence so get 1-to-1 correspondence of mesh face to skeletal piece
    local_correspondence_revised = correspondence_1_to_1(mesh=mesh,
                                    local_correspondence=local_correspondence,
                                    curr_limb_endpoints_must_keep=curr_limb_endpoints_must_keep,
                                    curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices)

    # -------3b) Fixing the mesh indices to correspond to the larger mesh as a whole
    for k,v in local_correspondence_revised.items():
        local_correspondence_revised[k]["branch_face_idx"] = mesh_idx[local_correspondence_revised[k]["branch_face_idx"]]

    print(f"Total time for MAP sublimb #{sublimb_idx} mesh processing = {time.time() - mesh_start_time}")

    if print_fusion_steps:
        print(f"correspondence_1_to_1: {time.time() - fusion_time }")
        fusion_time = time.time()

    limb_correspondence_MAP[sublimb_idx] = local_correspondence_revised
    
    raise Exception("Done #0 mesh piece")

print(f"Total time for MAP sublimb processing {time.time() - global_start_time}")

In [ ]:
current_skeleton = su.decompress_pickle("current_skeleton")

In [ ]:
nviz.plot_objects(debug_mesh,
                 skeletons=[current_skeleton])

# Do the new function where can protect the segments we want

In [ ]:
import networkx_utils as xu

In [ ]:
"""
Pseudocode: 
1) Turn the skeleton into a graph
2) Get all of the end nodes
3) Get all of the high degree nodes in the graph

if there are high degree nodes, for each end nodes
a. Find the path to the nearest high degree node
b. Get the skeleton of that subgraph
c. If length of that skeleton is within a certain range:
    1) Run the first pass mesh correspondence
    2) If the mesh is of a certain size, add the endpoint to the endpoints to keep

"""

In [ ]:
debug_mesh.submesh([[1]],append=True,repair=False)

In [ ]:

import trimesh_utils as tu
                                                                
return_value = tu.skeleton_to_mesh_correspondence( mesh = debug_mesh,
                                                skeletons = viable_end_node_skeletons
                                   )

nviz.plot_objects(meshes=return_value,
                  meshes_colors="random",
                  skeletons=viable_end_node_skeletons,
                 skeletons_colors="random")
    

In [ ]:
nviz.plot_objects(meshes=return_value,
                  meshes_colors="random",
                  skeletons=viable_end_node_skeletons,
                 skeletons_colors="random")

In [ ]:
skeleton = current_skeleton

In [ ]:
sk.find_end_nodes_with_significant_mesh_correspondence(
    mesh = debug_mesh,
    skeleton = current_skeleton,
    plot_viable_endpoint_correspondences = False,
    plot_keep_endpoints = False,
    verbose = False)

In [ ]:
viable_end_node_skeletons_len

In [ ]:
sk.skeletonize_connected_branch

In [ ]:
ret_col = nviz.visualize_neuron(retrieved_obj,
                      visualize_type=["mesh","skeleton"],
                     limb_branch_dict=dict(L1="all"),
                     return_color_dict=True)

In [ ]:
nviz.plot_limb_concept_network_2D(retrieved_obj,
                                 node_colors=ret_col,
                                 starting_soma=0,
                                 starting_soma_group=1)

In [ ]:
curr_limb = retrieved_obj[1]

In [ ]:
curr_limb.set_concept_network_directional?

In [ ]:
nru.all_soma_connnecting_endpionts_from_starting_info(network_starting_info_revised_cleaned)

In [ ]:
winning_vertex = np.array([1155881.9375  , 519313.2625  , 763731.68125])
nu.matching_rows(total_keep_endpoints,winning_vertex)

In [ ]:
limb_correspondence_individual = pre.filter_limb_correspondence_for_end_nodes(limb_correspondence=limb_correspondence_individual,
                                                 mesh=limb_mesh_mparty,
                                                 starting_info=network_starting_info_revised_cleaned,
                                                filter_end_node_length=filter_end_node_length

                                                )

In [ ]:
total_keep_endpoints

In [ ]:
st_coord = nru.all_soma_connnecting_endpionts_from_starting_info(network_starting_info_revised_cleaned)
st_coord

In [ ]:
st_coord

In [ ]:
total_keep_endpoints

In [ ]:
limb_to_endpoints_must_keep_list

In [ ]:
network_starting_info

In [ ]:
t_verts_list_total

In [ ]:
network_starting_info_revised

In [ ]:
network_starting_info_revised

In [ ]:
network_starting_info_revised[1]

In [ ]:
nru.all_soma_connnecting_endpionts_from_starting_info(network_starting_info_revised)

In [ ]:
nviz.plot_limb_correspondence(limb_correspondence_MP_saved,
                             meshes_colors="green",
                             skeleton_colors="blue",
                             scatters=total_keep_endpoints)

In [ ]:
nviz.plot_limb_correspondence(limb_correspondence_MAP_saved,
                             meshes_colors="green",
                             skeleton_colors="blue",
                             scatters=st_coord[0:1])

In [ ]:
nviz.plot_limb_correspondence(limb_correspondence_individual,
                             meshes_colors="green",
                             skeleton_colors="blue",
                             scatters=st_coord)

# Debugging the limb correspondence error

In [ ]:
for sublimb_idx,(mesh,mesh_idx) in enumerate(zip(mesh_pieces_for_MAP,mesh_pieces_for_MAP_face_idx)):
    print(f"--- Working on MAP piece {sublimb_idx}---")
    #print(f"soma_touching_vertices_dict = {soma_touching_vertices_dict}")
    mesh_start_time = time.time()
    curr_soma_to_piece_touching_vertices = filter_soma_touching_vertices_dict_by_mesh(
    mesh = mesh,
    curr_piece_to_soma_touching_vertices = soma_touching_vertices_dict
    )

    if print_fusion_steps:
        print(f"MAP Filtering Soma Pieces: {time.time() - fusion_time }")
        fusion_time = time.time()

    # ---- 0) Generating the Clean skeletons  -------------------------------------------#
    if not curr_soma_to_piece_touching_vertices is None:
        curr_total_border_vertices = dict([(k,np.vstack(v)) for k,v in curr_soma_to_piece_touching_vertices.items()])
    else:
        curr_total_border_vertices = None


    cleaned_branch,curr_limb_endpoints_must_keep = sk.skeletonize_and_clean_connected_branch_CGAL(
        mesh=mesh,
        curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices,
        total_border_vertices=curr_total_border_vertices,
        filter_end_node_length=filter_end_node_length,
        perform_cleaning_checks=perform_cleaning_checks,
        combine_close_skeleton_nodes = combine_close_skeleton_nodes,
        combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold,
    use_surface_after_CGAL=use_surface_after_CGAL,
    surface_reconstruction_size=surface_reconstruction_size)

    if not curr_limb_endpoints_must_keep is None:
        limb_to_endpoints_must_keep_list.append(curr_limb_endpoints_must_keep)
        limb_to_soma_touching_vertices_list.append(curr_soma_to_piece_touching_vertices)
    else:
        print("Inside MAP decomposition and curr_limb_endpoints_must_keep was None")

    if len(cleaned_branch) == 0:
        raise Exception(f"Found a zero length skeleton for limb {z} of trmesh {branch}")

    if print_fusion_steps:
        print(f"skeletonize_and_clean_connected_branch_CGAL: {time.time() - fusion_time }")
        fusion_time = time.time()

    # ---- 1) Generating Initial Mesh Correspondence -------------------------------------------#
    start_time = time.time()

    print(f"Working on limb correspondence for #{sublimb_idx} MAP piece")
    local_correspondence = mesh_correspondence_first_pass(mesh=mesh,
                                                         skeleton=cleaned_branch,
                                                         distance_by_mesh_center=distance_by_mesh_center,
                                                         connectivity="edges",
                                                         remove_inside_pieces_threshold=100)


    print(f"Total time for decomposition = {time.time() - start_time}")
    if print_fusion_steps:
        print(f"mesh_correspondence_first_pass: {time.time() - fusion_time }")
        fusion_time = time.time()


    #------------- 2) Doing Some checks on the initial corespondence -------- #


    if perform_cleaning_checks:
        check_skeletonization_and_decomp(skeleton=cleaned_branch,
                                        local_correspondence=local_correspondence)

    # -------3) Finishing off the face correspondence so get 1-to-1 correspondence of mesh face to skeletal piece
    local_correspondence_revised = correspondence_1_to_1(mesh=mesh,
                                    local_correspondence=local_correspondence,
                                    curr_limb_endpoints_must_keep=curr_limb_endpoints_must_keep,
                                    curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices)

    # -------3b) Fixing the mesh indices to correspond to the larger mesh as a whole
    for k,v in local_correspondence_revised.items():
        local_correspondence_revised[k]["branch_face_idx"] = mesh_idx[local_correspondence_revised[k]["branch_face_idx"]]

    print(f"Total time for MAP sublimb #{sublimb_idx} mesh processing = {time.time() - mesh_start_time}")

    if print_fusion_steps:
        print(f"correspondence_1_to_1: {time.time() - fusion_time }")
        fusion_time = time.time()

    limb_correspondence_MAP[sublimb_idx] = local_correspondence_revised
    
    raise Exception("Done #0 mesh piece")

print(f"Total time for MAP sublimb processing {time.time() - global_start_time}")

In [ ]:
nviz.plot_objects(mesh,
                 skeletons=[cleaned_branch])

In [ ]:
nviz.plot_limb_correspondence(local_correspondence,mesh_name="correspondence_mesh")

In [ ]:
nviz = reload(nviz)
nviz.plot_limb_correspondence(local_correspondence,mesh_name="correspondence_mesh")

In [ ]:
nviz.plot_limb_correspondence(local_correspondence_revised)

# Debugging the local correspondence

In [ ]:
mesh=mesh
skeleton=cleaned_branch
distance_by_mesh_center=distance_by_mesh_center
connectivity="edges"

skeleton_branches=None

"""
Will come up with the mesh correspondences for all of the skeleton
branches: where there can be overlaps and empty faces

"""
curr_limb_mesh = mesh
curr_limb_sk = skeleton

if skeleton_branches is None:
    if skeleton is None:
        raise Exception("Both skeleton and skeleton_branches is None")
    curr_limb_branches_sk_uneven = sk.decompose_skeleton_to_branches(curr_limb_sk) #the line that is decomposing to branches
else:
    curr_limb_branches_sk_uneven = skeleton_branches 

#Doing the limb correspondence for all of the branches of the skeleton
local_correspondence = dict()
for j,curr_branch_sk in tqdm(enumerate(curr_limb_branches_sk_uneven)):
    local_correspondence[j] = dict()


    returned_data = cu.mesh_correspondence_adaptive_distance(curr_branch_sk,
                                  curr_limb_mesh,
                                 skeleton_segment_width = 1000,
                                 distance_by_mesh_center=distance_by_mesh_center,
                                                            connectivity=connectivity)
    if len(returned_data) == 0:
        print("Got nothing from first pass so expanding the mesh correspondnece parameters ")
        returned_data = cu.mesh_correspondence_adaptive_distance(curr_branch_sk,
                                  curr_limb_mesh,
                                 skeleton_segment_width = 1000,
                                 distance_by_mesh_center=distance_by_mesh_center,
                                buffer=300,
                                 distance_threshold=6000,
                                return_closest_face_on_empty=True,
                                    connectivity=connectivity)

    # Need to just pick the closest face is still didn't get anything

    # ------ 12/3 Addition: Account for correspondence that does not work so just picking the closest face
    curr_branch_face_correspondence, width_from_skeleton = returned_data


#             print(f"curr_branch_sk.shape = {curr_branch_sk.shape}")
#             np.savez("saved_skeleton_branch.npz",curr_branch_sk=curr_branch_sk)
#             tu.write_neuron_off(curr_limb_mesh,"curr_limb_mesh.off")
#             #print(f"returned_data = {returned_data}")
#             raise Exception(f"The output from mesh_correspondence_adaptive_distance was nothing: curr_branch_face_correspondence")


    if len(curr_branch_face_correspondence) > 0:
        curr_submesh = curr_limb_mesh.submesh([list(curr_branch_face_correspondence)],append=True,repair=False)
    else:
        curr_submesh = trimesh.Trimesh(vertices=np.array([]),faces=np.array([]))


    local_correspondence[j]["branch_skeleton"] = curr_branch_sk
    local_correspondence[j]["correspondence_mesh"] = curr_submesh
    local_correspondence[j]["correspondence_face_idx"] = curr_branch_face_correspondence
    local_correspondence[j]["width_from_skeleton"] = width_from_skeleton

In [ ]:
tu.original_mesh_faces_map(curr_limb_mesh,inside_pieces)

In [ ]:
curr_limb_mesh,len(new_mesh)

In [ ]:
new_mesh = tu.remove_mesh_interior(curr_limb_mesh,size_threshold_to_remove=100,try_hole_close=False,return_removed_pieces=False,
                                  return_face_indices=True)

nviz.plot_objects(curr_limb_mesh.submesh([new_mesh],append=True))

In [ ]:
test_skeleton = curr_limb_branches_sk_uneven[2]
curr_branch_face_correspondence, width_from_skeleton = cu.mesh_correspondence_adaptive_distance(test_skeleton,
                                 new_mesh,
                                 skeleton_segment_width = 1000,
                                 distance_by_mesh_center=distance_by_mesh_center,
                                                            connectivity=connectivity,
                                                                                               #buffer=3000
                                                                                               )

In [ ]:
curr_submesh = new_mesh.submesh([list(curr_branch_face_correspondence)],append=True,repair=False)
nviz.plot_objects(curr_submesh,
                 skeletons=[test_skeleton],
                 meshes=inside_pieces,
                 meshes_colors="red")

In [ ]:
len(face_subtract_indices)

In [ ]:
face_subtract_indices = su.decompress_pickle("face_subtract_indices")
debug_mesh = curr_limb_mesh.submesh([np.unique(np.concatenate(face_subtract_indices))],append=True,)

In [ ]:
ret_val = tu.remove_mesh_interior(debug_mesh,size_threshold_to_remove=100,verbose=True,return_removed_pieces=True)

In [ ]:
debug_mesh_revised,inside_pieces = ret_val

In [ ]:
nviz.plot_objects(main_mesh=debug_mesh_revised,
                meshes=inside_pieces,
                 meshes_colors="random")

In [ ]:
split_meshes = tu.split_significant_pieces(debug_mesh,connectivity="vertices")
nviz.plot_objects(meshes=split_meshes,
                  meshes_colors="random")

In [ ]:
nviz.plot_objects(meshes=tu.split(debug_mesh,connectivity="edges")[0],
                  meshes_colors="random")

In [ ]:
nviz.plot_objects(,
                  skeletons=[test_skeleton])
                  


In [ ]:
face_subtract_indices = su.decompress_pickle("face_subtract_indices")
conn_face_components = nu.intersecting_array_components(face_subtract_indices,sort_components=True)
conn_comps_lenghts = np.array([len(k) for k in conn_face_components])
max_len_components_idx = np.where(conn_comps_lenghts == np.max(conn_comps_lenghts))[0]

#2) Get the face indices for all those in the group
max_len_components_unique_faces = [np.unique(np.concatenate([face_subtract_indices[k] 
                                                             for k in conn_face_components[cmp_idx]])) for cmp_idx in  max_len_components_idx]
unique_removed_faces_pre = max_len_components_unique_faces[np.argmax([len(k) for k in max_len_components_unique_faces])]
unique_removed_faces_pre

In [ ]:
nviz.plot_objects(curr_limb_mesh.submesh([unique_removed_faces_pre],append=True),
                 skeletons=[test_skeleton])

In [ ]:
nviz.plot_objects(skeletons=curr_limb_branches_sk_uneven,
                 scatters=[curr_limb_branches_sk_uneven[2].reshape(-1,3)])

In [ ]:
limb_correspondence_individual[0]

In [ ]:
dict([(k,v) for k,v in limb_correspondence_individual.items() if k in np.arange(10,20)])

In [ ]:
nviz.plot_limb_correspondence(dict([(k,v) for k,v in limb_correspondence_individual.items() if k in np.arange(19,20)]))

In [ ]:
nviz.plot_objects(meshes=mesh_pieces_for_MAP,
                  meshes_colors=["red","black"])

In [ ]:
nviz.plot_limb_correspondence(limb_correspondence_MAP)

In [ ]:
nviz.visualize_neuron(
                        recovered_neuron,
                      visualize_type=["mesh","skeleton"],
                      limb_branch_dict=dict(L1="all")
                     )

In [ ]:
curr_br_endpts_unique

In [ ]:
nviz.plot_objects(skeletons=[curr_skeleton_MAP,curr_skeleton_MP],
                 skeletons_colors=["red","black"],
                 scatters=[MAP_stitch_point.reshape(-1,3)],
                 scatter_size=0.4)

In [ ]:
curr_br_endpts_unique

In [ ]:
nu.setdiff2d(curr_br_endpts_unique,total_keep_endpoints)

In [ ]:
st = time.time()
np.array([k for k in curr_br_endpts_unique if len(nu.matching_rows(total_keep_endpoints,k))==0])
print(f"total time = {time.time() - st}")

In [ ]:
st = time.time()
np.array([k for k in curr_br_endpts_unique if len(nu.matching_rows(total_keep_endpoints,k))==0])
print(f"total time = {time.time() - st}")

In [ ]:
sk = reload(sk)
sk.move_point_to_nearest_branch_end_point_within_threshold(
                                                    skeleton=curr_skeleton_MAP,
                                                    coordinate=MAP_stitch_point,
                                                    distance_to_move_point_threshold = distance_to_move_point_threshold,
                                                    verbose=True,
                                                    possible_node_coordinates=nu.setdiff2d(curr_br_endpts_unique,total_keep_endpoints),
                                                    excluded_node_coordinates=total_keep_endpoints,
                                                    )

In [ ]:
network_starting_info_revised_cleaned

In [ ]:
st_endpts = nru.all_soma_connnecting_endpionts_from_starting_info(network_starting_info_revised_cleaned,multiple_limbs=False)
meshes,skeletons = nviz.limb_correspondence_plottable(limb_correspondence_MAP_saved)
nviz.plot_objects(meshes=meshes,
                  skeletons=skeletons,
                             meshes_colors="green",
                             skeletons_colors="blue",
                 scatters=[st_endpts],
                 scatter_size=0.5)

In [ ]:
meshes,skeletons = nviz.limb_correspondence_plottable(limb_correspondence_individual)

In [ ]:
total_skeleton = sk.stack_skeletons(skeletons)

In [ ]:
nviz.plot_objects(
                #meshes=meshes,
                skeletons=[skeletons[k] for k in [1,86]],
                 skeletons_colors="random",
                scatters=[skeletons[1].reshape(-1,3)],
    scatter_size=1
)

In [ ]:
sk.calculate_skeleton_distance(skeletons[1])

In [ ]:
sk.calculate_skeleton_distance(skeletons[86])

In [ ]:
sk.find_branch_skeleton_with_specific_coordinate(skeletons,st_endpts[2])

In [ ]:
st_endpts = nru.all_soma_connnecting_endpionts_from_starting_info(network_starting_info_revised_cleaned,multiple_limbs=False)
meshes,skeletons = nviz.limb_correspondence_plottable(limb_correspondence_individual)
nviz.plot_objects(meshes=meshes,
                  skeletons=skeletons,
                             meshes_colors="green",
                             skeletons_colors="blue",
                 scatters=[st_endpts[2]],
                 scatter_size=0.5)

In [ ]:

st_endpts

In [ ]:
meshes,skeletons = nviz.limb_correspondence_plottable(limb_correspondence_MAP_saved)
nviz.plot_objects(meshes=meshes,
                  skeletons=skeletons,
                             meshes_colors="green",
                             skeletons_colors="blue",
                 scatters=[st_endpts],
                 scatter_size=0.1)

In [ ]:
nviz.plot_limb_correspondence(limb_correspondence_MP,
                             meshes_colors="green",
                             skeleton_colors="blue")

In [ ]:
"""
Big Conclusion: the large mesh pieces themselves (before combining into map pieces)
themselves aren't totally connected by edges (can be split)

- so even if large pieces do have a shared edge and you combine them together,
they can still be split by the edges into multiple pieces

Solution: should have to do split by vertices in the skeletonization

"""

In [ ]:
for sublimb_idx,(mesh,mesh_idx) in enumerate(zip(mesh_pieces_for_MAP,mesh_pieces_for_MAP_face_idx)):
    print(f"--- Working on MAP piece {sublimb_idx}---")
    #print(f"soma_touching_vertices_dict = {soma_touching_vertices_dict}")
    mesh_start_time = time.time()
    curr_soma_to_piece_touching_vertices = filter_soma_touching_vertices_dict_by_mesh(
    mesh = mesh,
    curr_piece_to_soma_touching_vertices = soma_touching_vertices_dict
    )

    if print_fusion_steps:
        print(f"MAP Filtering Soma Pieces: {time.time() - fusion_time }")
        fusion_time = time.time()

    # ---- 0) Generating the Clean skeletons  -------------------------------------------#
    if not curr_soma_to_piece_touching_vertices is None:
        curr_total_border_vertices = dict([(k,np.vstack(v)) for k,v in curr_soma_to_piece_touching_vertices.items()])
    else:
        curr_total_border_vertices = None


    cleaned_branch,curr_limb_endpoints_must_keep = sk.skeletonize_and_clean_connected_branch_CGAL(
        mesh=mesh,
        curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices,
        total_border_vertices=curr_total_border_vertices,
        filter_end_node_length=filter_end_node_length,
        perform_cleaning_checks=perform_cleaning_checks,
        combine_close_skeleton_nodes = combine_close_skeleton_nodes,
        combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold,
    use_surface_after_CGAL=use_surface_after_CGAL,
    surface_reconstruction_size=surface_reconstruction_size,
    skeleton_print=True,
    connectivity="vertices"
    )
    
    raise Exception("")

In [ ]:
nviz.plot_objects(meshes=[mesh_pieces_for_MAP[0]],
                 skeletons=[cleaned_branch])

In [ ]:
current_mesh_splits = tu.split_significant_pieces(mesh_pieces_for_MAP[0],
                               significance_threshold=1,
                                                  connectivity="vertices")
current_mesh_splits

In [ ]:
tu.